In [19]:
from pathlib import Path
import sys
 
tiramisu_filesys = '/data/mm12191/'
benchmark_dataset_file = tiramisu_filesys+'datasets/benchmarks_ds2.json'

store_device_name = 'cuda:4'
train_device_name = 'cuda:4'

k=5

### You should execute all cells after 'Utils' cell below, in order to continue the execution of this notebook. Good luck!

### Load Data
<a id='load_data'></a>

In [71]:
#loading data for the cost model
bench_ds, bench_bl, bench_indices, _, _ = load_data('/data/mm12191/datasets/benchmarks_ds2.json', 
                                             split_ratio = 1,
                                             max_batch_size = 1,
                                             drop_sched_func = drop_schedule, 
                                             drop_prog_func = None,
                                             default_eval = default_eval,
                                             speedups_clip_func = speedup_clip)
#loading data for the k-best LI model
bench_ds_LI, bench_bl_LI, bench_indices_LI, _, _  = load_merge_data(None, benchmark_dataset_file, 1, filter_func_MC=filter_schedule_MC, filter_func_SC=filter_schedule_SC)

loading batches from: /data/mm12191/datasets/benchmarks_ds2.json


  0%|          | 0/30 [00:00<?, ?it/s]

Batching ... 0


  0%|          | 0/3 [00:00<?, ?it/s]

Number of datapoints 8884 Number of batches 8884
Data loaded
Sizes: (8884, 0) batches


  0%|          | 0/30 [00:00<?, ?it/s]

Number of batches 3
Data loaded
Sizes: (3, 0) batches


### Define Models

In [72]:
#speedup model
input_size = 776

model = None 

model = Model_Recursive_LSTM_v2(input_size,drops=[0.250, 0.250, 0.250, 0.250])
model.to(train_device)

criterion = mape_criterion

optimizer = AdamW(model.parameters(),weight_decay=0.1e-1)    
    
# model = model.double()

In [73]:
#LI model
input_size = 1272  #1238 #739 + 1 (loop depth added) precedently

LI_model = None 

#model = Model_Recursive_LSTM_v2(input_size,drops=[0.112, 0.112, 0.112, 0.112], output_size=106) /
# model = Model_Recursive_LSTM_v2(input_size,comp_embed_layer_sizes=[600, 350, 200, 180], drops=[0.4, 0.175, 0.275, 0.175], output_size=106) #first
LI_model = Model_Recursive_LSTM_LI(input_size,comp_embed_layer_sizes=[600, 900, 600, 400, 200], drops=[0.275, 0.4, 0.275, 0.175, 0.175], output_size=106 * 5)

LI_model.to(train_device) 

# criterion = nn.CrossEntropyLoss()

# optimizer = AdamW(model.parameters(),weight_decay=0.375e-2)  #0.375e-2
# optimizer = optim.RMSprop(model.parameters(), weight_decay=0.3e-2)    # RMSprop, Adagrad


Model_Recursive_LSTM_LI(
  (comp_embedding_layers): ModuleList(
    (0): Linear(in_features=1272, out_features=600, bias=True)
    (1): Linear(in_features=600, out_features=900, bias=True)
    (2): Linear(in_features=900, out_features=600, bias=True)
    (3): Linear(in_features=600, out_features=400, bias=True)
    (4): Linear(in_features=400, out_features=200, bias=True)
  )
  (comp_embedding_dropouts): ModuleList(
    (0): Dropout(p=0.275, inplace=False)
    (1): Dropout(p=0.4, inplace=False)
    (2): Dropout(p=0.275, inplace=False)
    (3): Dropout(p=0.175, inplace=False)
    (4): Dropout(p=0.175, inplace=False)
  )
  (regression_layers): ModuleList(
    (0): Linear(in_features=200, out_features=400, bias=True)
    (1): Linear(in_features=400, out_features=200, bias=True)
  )
  (regression_dropouts): ModuleList(
    (0): Dropout(p=0.275, inplace=False)
    (1): Dropout(p=0.4, inplace=False)
  )
  (concat_layers): ModuleList(
    (0): Linear(in_features=424, out_features=400, bias=Tr

### Load Models

In [74]:
model.load_state_dict(torch.load('/data/lm4718/CostModelMulti/MAPE_base_13+4+2.6.pkl',map_location=train_device))

#not the last one
# model.load_state_dict(torch.load('/data/mm12191/model_lab/temps_save_simple_fixed_val_batch550000-716507_STUE_ScedP_DefEval.pkl',map_location=train_device))
model.to(train_device)
print()

In [75]:
# LI_model.load_state_dict(torch.load('/data/lm4718/SC_ALL_Adams_ELU_xavier_13M_KBEST_2.pkl',map_location=train_device))
# LI_model.load_state_dict(torch.load('/data/lm4718/LI_ALL_BEST_K/Models/SC_ALL_Adams_ELU_xavier_13M_KBEST_df_2.pkl',map_location=train_device))
LI_model.load_state_dict(torch.load('/data/lm4718/LI_k_full/Models/full_1000_0.001_106_13M115_dropout_fused_kBest_0.0006_2000_4.pkl',map_location=train_device))
LI_model.to(train_device)

print()

### Results 

In [76]:
bench_df = get_results_df(bench_ds, bench_bl, bench_indices, model)
LI_bench_df = get_results_df_LI(bench_ds_LI, bench_bl_LI, bench_indices_LI, LI_model)

  0%|          | 0/8884 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

### Exemple of the LI results output 

In [79]:
LI_bench_df.iloc[0]["target"]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [80]:
LI_bench_df.iloc[0]["prediction"]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [82]:
#extract the LI from the output vectors
i = 12
get_LI_ordered(LI_bench_df.iloc[i]["prediction"]), get_LI_ordered(LI_bench_df.iloc[i]["target"])

(['', 'I(L0,L1)', 'I(L1,L2)', 'I(L0,L2)'],
 ['I(L0,L1)', '', 'I(L0,L2)', 'I(L1,L2)'])

#### Simulate beam search

In [83]:
#get the predictions made by the LI model to use them in the Simulation of beam search.
enforced_scheds_df = pd.DataFrame(columns=['name','sched_str', 'depth','priority'])
enf = 0
for ind in LI_bench_df.index:
    name = LI_bench_df["name"][ind]
    LIs = get_LI_ordered(LI_bench_df["prediction"][ind])
    if LIs == []:
        print("--")
    i = 0
    for li in LIs:
        enforced_scheds_df.loc[enf] = [name, li, 1, i]# 1 is the level
        i += 1
        enf += 1
enforced_scheds_df

,name,sched_str,depth,priority
0,function_heat2d_LARGE,,1,0
1,function_heat2d_LARGE,"I(L0,L1)",1,1
2,function_heat2d_MEDIUM,,1,0
3,function_heat2d_MEDIUM,"I(L0,L1)",1,1
4,function_heat2d_MINI,,1,0
5,function_heat2d_MINI,"I(L0,L1)",1,1
6,function_heat2d_SMALL,,1,0
7,function_heat2d_SMALL,"I(L0,L1)",1,1
8,function_heat2d_XLARGE,,1,0
9,function_heat2d_XLARGE,"I(L0,L1)",1,1


In [56]:
#use only the cost model
# enforced_scheds_df = pd.DataFrame(columns=['name','sched_str', 'depth'])
# enforced_scheds_df

,name,sched_str,depth


In [92]:
#return the results of beam search, with the speedups of the final schedules
pd.set_option('display.max_rows',100)
# df = simulate_BeamSearch_on_Dataset(bench_ds,bench_df, enforced_scheds_df, true_beam_search=False, get='schedules')
df = simulate_BeamSearch_on_Dataset(bench_ds,bench_df, enforced_scheds_df, true_beam_search=False, get='speedups')
df

  0%|          | 0/30 [00:00<?, ?it/s]

,name,nb_scheds,base_time,eval_mode,bs=1,bs=2,bs=3,bs=4,bs=5,bs=6,bs=7,bs=8,bs=9
0,function_seidel2d_SMALL,41,7.01036,execution,1.26,1.26,1.43,1.43,1.43,1.43,1.43,1.43,1.43
1,function_seidel2d_SMALL,41,7.01036,model,1.26,1.26,1.43,1.43,1.43,1.43,1.43,1.43,1.43
2,function_seidel2d_SMALL,41,7.01036,level1,1.26,1.26,1.43,1.43,1.43,1.43,1.43,1.43,1.43
3,function_seidel2d_MINI,35,0.835829,execution,1.47,1.47,1.47,1.47,1.47,1.61,1.62,1.64,1.64
4,function_seidel2d_MINI,35,0.835829,model,1.47,1.47,1.47,1.47,1.47,1.61,1.62,1.64,1.64
5,function_seidel2d_MINI,35,0.835829,level1,1.47,1.47,1.47,1.47,1.47,1.61,1.62,1.64,1.64
6,function_seidel2d_MEDIUM,227,232.917999,execution,1.49,1.56,1.56,1.56,3.40,3.40,3.40,3.40,3.40
7,function_seidel2d_MEDIUM,227,232.917999,model,1.49,1.56,1.56,1.56,3.40,3.40,3.40,3.40,3.40
8,function_seidel2d_MEDIUM,227,232.917999,level1,1.49,1.56,1.56,1.56,3.40,3.40,3.40,3.40,3.40
9,function_seidel2d_LARGE,359,19681.5,execution,12.54,14.89,14.89,14.89,14.89,14.89,14.89,14.89,14.89


In [93]:
#return the results of beam search with its final schedules
pd.set_option('display.max_rows',100)
# df = simulate_BeamSearch_on_Dataset(bench_ds,bench_df, enforced_scheds_df, true_beam_search=False, get='schedules')
df = simulate_BeamSearch_on_Dataset(bench_ds,bench_df, enforced_scheds_df, true_beam_search=False, get='schedules')
df

  0%|          | 0/30 [00:00<?, ?it/s]

,name,nb_scheds,base_time,eval_mode,bs=1,bs=2,bs=3,bs=4,bs=5,bs=6,bs=7,bs=8,bs=9
0,function_seidel2d_SMALL,41,7.01036,execution,"S(L1,L2,3,2)","S(L1,L2,3,2)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)"
1,function_seidel2d_SMALL,41,7.01036,model,"S(L1,L2,3,2)","S(L1,L2,3,2)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)"
2,function_seidel2d_SMALL,41,7.01036,level1,"S(L1,L2,3,2)","S(L1,L2,3,2)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)","S(L1,L2,2,1)T2(L1,L2,32,32)"
3,function_seidel2d_MINI,35,0.835829,execution,"S(L1,L2,3,1)","S(L1,L2,3,1)","S(L1,L2,3,1)","S(L1,L2,3,1)","S(L1,L2,3,1)","S(L0,L1,2,1)U(L2,8)","S(L0,L1,3,2)U(L2,16)","S(L0,L1,3,1)U(L2,8)","S(L0,L1,3,1)U(L2,8)"
4,function_seidel2d_MINI,35,0.835829,model,"S(L1,L2,3,1)","S(L1,L2,3,1)","S(L1,L2,3,1)","S(L1,L2,3,1)","S(L1,L2,3,1)","S(L0,L1,2,1)U(L2,8)","S(L0,L1,3,2)U(L2,16)","S(L0,L1,3,1)U(L2,8)","S(L0,L1,3,1)U(L2,8)"
5,function_seidel2d_MINI,35,0.835829,level1,"S(L1,L2,3,1)","S(L1,L2,3,1)","S(L1,L2,3,1)","S(L1,L2,3,1)","S(L1,L2,3,1)","S(L0,L1,2,1)U(L2,8)","S(L0,L1,3,2)U(L2,16)","S(L0,L1,3,1)U(L2,8)","S(L0,L1,3,1)U(L2,8)"
6,function_seidel2d_MEDIUM,227,232.917999,execution,"S(L1,L2,3,1)","S(L1,L2,3,2)T2(L1,L2,32,64)","S(L1,L2,3,2)T2(L1,L2,32,64)","S(L1,L2,3,2)T2(L1,L2,32,64)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)"
7,function_seidel2d_MEDIUM,227,232.917999,model,"S(L1,L2,3,1)","S(L1,L2,3,2)T2(L1,L2,32,64)","S(L1,L2,3,2)T2(L1,L2,32,64)","S(L1,L2,3,2)T2(L1,L2,32,64)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)"
8,function_seidel2d_MEDIUM,227,232.917999,level1,"S(L1,L2,3,1)","S(L1,L2,3,2)T2(L1,L2,32,64)","S(L1,L2,3,2)T2(L1,L2,32,64)","S(L1,L2,3,2)T2(L1,L2,32,64)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)","S(L0,L1,2,1)P(L1)T2(L0,L1,128,32)U(L4,16)"
9,function_seidel2d_LARGE,359,19681.5,execution,"S(L0,L1,3,2)P(L1)","S(L0,L1,2,1)P(L1)U(L2,4)","S(L0,L1,2,1)P(L1)U(L2,4)","S(L0,L1,2,1)P(L1)U(L2,4)","S(L0,L1,2,1)P(L1)U(L2,4)","S(L0,L1,2,1)P(L1)U(L2,4)","S(L0,L1,2,1)P(L1)U(L2,4)","S(L0,L1,2,1)P(L1)U(L2,4)","S(L0,L1,2,1)P(L1)U(L2,4)"


In [91]:
search_perf_df = get_search_performance(bench_ds,bench_df, enforced_scheds_df,true_beam_search=False, tira = False)
search_perf = np.mean([min(100,i) for i in search_perf_df['bs=3']])
np.mean(search_perf_df['bs=1']), np.mean(search_perf_df['bs=2']), np.mean(search_perf_df['bs=3']), np.mean(search_perf_df['bs=4']), search_perf

  0%|          | 0/30 [00:00<?, ?it/s]

(93.48911488557209, 97.59062572154238, 104.25396825396825, 100.0, 100.0)

In [79]:
search_perf_df

,name,nb_scheds,base_time,tira,bs=1,bs=2,bs=3,bs=4,bs=5,bs=6,bs=7,bs=8,bs=9
0,function_seidel2d_SMALL,41,7.01036,True,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
1,function_seidel2d_MINI,35,0.835829,True,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
2,function_seidel2d_MEDIUM,227,232.917999,True,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
3,function_seidel2d_LARGE,359,19681.5,True,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
4,function_matmul_SMALL,762,0.946007,True,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
5,function_matmul_MINI,99,0.026936,True,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
6,function_matmul_MEDIUM,3432,33.081902,True,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
7,function_matmul_LARGE,4692,16346.0,True,15.542591,38.212164,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
8,function_jacobi2d_SMALL,100,7.21077,True,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
9,function_jacobi2d_MINI,70,1.24685,True,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0,100.0,100.0


In [80]:
search_perf_df = get_search_performance(bench_ds,bench_df, enforced_scheds_df,true_beam_search=False, tira = False)
search_perf = np.mean([min(100,i) for i in search_perf_df['bs=3']])
np.mean(search_perf_df['bs=2']) , np.mean(search_perf_df['bs=3']), np.mean(search_perf_df['bs=4']), search_perf

  0%|          | 0/31 [00:00<?, ?it/s]

(100.0, 104.11674347158218, 100.0, 100.0)

In [81]:
search_perf_df

,name,nb_scheds,base_time,tira,bs=1,bs=2,bs=3,bs=4,bs=5,bs=6,bs=7,bs=8,bs=9
0,function_seidel2d_SMALL,41,7.01036,False,100.000000,100.0,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
1,function_seidel2d_MINI,35,0.835829,False,100.000000,100.0,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
2,function_seidel2d_MEDIUM,227,232.917999,False,100.000000,100.0,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
3,function_seidel2d_LARGE,359,19681.5,False,100.000000,100.0,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
4,function_matmul_SMALL,762,0.946007,False,100.000000,100.0,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
5,function_matmul_MINI,99,0.026936,False,100.000000,100.0,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
6,function_matmul_MEDIUM,3432,33.081902,False,100.000000,100.0,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
7,function_matmul_LARGE,4692,16346.0,False,100.000000,100.0,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
8,function_jacobi2d_SMALL,100,7.21077,False,100.000000,100.0,100.000000,100.0,100.0,100.0,100.0,100.0,100.0
9,function_jacobi2d_MINI,70,1.24685,False,100.000000,100.0,100.000000,100.0,100.0,100.0,100.0,100.0,100.0


# Utils
<a id='utils'></a>

In [4]:
import ujson as json
from tqdm.notebook import tqdm
import re
import torch
import numpy as np
import random 
import torch.nn as nn
import pandas as pd
from torch import optim
import time
import sys
import pickle
try:
    from torch.optim import AdamW
    from torch.optim.lr_scheduler import OneCycleLR
except:
    sys.path.append("../_new_repr_to_reorder/2_Model_training_tests/CostModels/")
    from src.torch141.lr_scheduler import *
    from src.torch141.adamw import *
from IPython.display import clear_output
from matplotlib import pyplot as plt
import copy
import math
from scipy.stats import spearmanr
from sklearn.metrics import ndcg_score


store_device = torch.device(store_device_name)
train_device = torch.device(train_device_name)



class NbAccessException(Exception):
    pass
class LoopsDepthException(Exception):
    pass

#### Functions for dataloading

In [69]:
#for cost model
def speedup_clip(speedup):
    if speedup<0.01:
        speedup = 0.01
    return speedup

def drop_program(prog_dict):   
    if len(prog_dict['schedules_list'])<2:
        return True
    if has_skippable_loop_1comp(prog_dict):
        return True
    if 'node_name' in prog_dict and prog_dict['node_name']=='lanka24': # drop if we the program is run by lanka24 (because its measurements are inacurate)
        return True
    return False   

def drop_schedule(prog_dict, schedule_index):
    schedule_json =  prog_dict['schedules_list'][schedule_index]
    schedule_str = sched_json_to_sched_str(schedule_json)
    program_depth = len(prog_dict['program_annotation']['iterators'])
    if (not schedule_json['execution_times']) or min(schedule_json['execution_times'])<0: # exec time is set to -1 on datapoints that are deemed noisy, or if list empty
        return True
    if len(prog_dict['program_annotation']['computations'])==1: #this function works only on single comp programs
        if sched_is_prunable_1comp(schedule_str,program_depth):
            return True 
    
    if len(schedule_json['execution_times'])==1:
        total_def_eval+=1
        if 'function760518'<prog_dict['filename'][2:16]<'function761289': 
            def_eval_in_range+=1
        if total_def_eval%10==0:
            print(total_def_eval,def_eval_in_range)
        return True
    if wrongly_pruned_schedule(prog_dict, schedule_index):
        return True

    return False

def default_eval(prog_dict, schedule_index):
    schedule_json =  prog_dict['schedules_list'][schedule_index]
    schedule_str = sched_json_to_sched_str(schedule_json)
    program_depth = len(prog_dict['program_annotation']['iterators'])
    if len(prog_dict['program_annotation']['computations'])==1: #this function works only on single comp programs
        return can_set_default_eval_1comp(schedule_str,program_depth)
    else:
        return 0

In [70]:
#for LI
def filter_schedule_MC(schedule_str): # needs to return True if we want the passed schedule to be dropped 
    
    regex1 = "^I\(\{[C0-9,]+\},L[0-9]+,L[0-9]+\)$"
    regex2 = "F\(\{[C0-9,]+\},L[0-9]+\)I\(\{[C0-9,]+\},L[0-9]+,L[0-9]+\)$"
    if re.search(regex1, schedule_str) or re.search(regex2, schedule_str) : # drops all the schedules except the loop interchanges
        return True
    if schedule_str=="":
        return True
    return False

def filter_schedule_SC(schedule_str): # needs to return True if we want the passed schedule to be dropped 
    
    regex = "I\(L[0-9]+,L[0-9]+\)$"
    #regex = "^I\(\{[C0-9,]+\},L[0-9]+,L[0-9]+\)$"
    if re.search(regex, schedule_str): # drops all the schedules except the loop interchanges
        return True
    if schedule_str=="":
        return True
    return False

#### Functions for Beam Search simulation

In [85]:
def update_exploration_trace(node, func_predictions_df, func_enforced_df):
    if not node['schedule'] in list(func_predictions_df['sched_str']):
        print(node['schedule'])
        return None
    sched = node['schedule']
    depth = node['depth']
    level_enforcement = func_enforced_df.query('depth==@depth') # it returns all the enforced nodes in the level

    if not level_enforcement.empty: #there is at least one enforced node in the level.
        node['level_has_enforcement'] = True
        res = level_enforcement.query('sched_str==@sched') 
        if not res.empty:
            node['enforced'] = True
            node['priority']= res.iloc[0]['priority']
        else:
            node['enforced'] = False
            node['priority']= 5 #because k = 5
        
    else:
        node['level_has_enforcement'] = False
        
    
#     node['id'] = int(func_predictions_df.query('sched_str == @sched')['sched_name'])
    new_children = []
    for child in node['children']:
        updated = update_exploration_trace(child, func_predictions_df, func_enforced_df)
        if updated != None:
            new_children.append(updated)
    node['children'] = new_children
    return node
  
def simulate_BeamSearch(root,beam_size,preds_dict=None,eval_mode='execution'): # given an exploration trace and a beam size, return the best candidate that can be found using that beam size
    children = root['children']
    if len(children)==0:
        return root
    
    if eval_mode != 'execution':
        if (children[0]['level_has_enforcement']): #that s our target level.
            if eval_mode == 'model':
                for child in children:
                    child['prediction'] = preds_dict[child['schedule']] #new column
                children = sorted(children, key = lambda x: x['prediction'], reverse=True) #was reverse = \true // maybe because it is an exec time
                children = children[:beam_size]
            else: #if level1
                assert children[0]['level_has_enforcement'] == True # if some enforcement is set at this level = special model
                children = sorted(children, key = lambda x: x['priority']) #take all the children, from small priority to highest one.
                children = children[:beam_size]
#                 print(children[0]['schedule'])
            root['prediction'] = preds_dict[root['schedule']] #do we need this ?
        else:
            children = sorted(children, key = lambda x: x['evaluation']) 
            children = children[:beam_size] 
#             print(children[0]['schedule'])
    else:
        children = sorted(children, key = lambda x: x['evaluation']) 
        children = children[:beam_size]  
#     print('----------------------------------------')
    bests = []
    for child in children:
        bests.append(simulate_BeamSearch(child,beam_size,preds_dict,eval_mode))
    bests.append(root) 
#     if eval_mode == 'model':
#         return max(bests, key = lambda x: x['prediction'])
#     elif eval_mode == 'execution':
#         return min(bests, key = lambda x: x['evaluation'])
    return min(bests, key = lambda x: x['evaluation'])
    
def simulate_TrueBeamSearch(root,beam_size,preds_dict=None,eval_mode='execution'): # given an exploration trace and a beam size, return the best candidate that can be found using that beam size
    if eval_mode == 'model':
        root['prediction'] = preds_dict[root['schedule']]
    candidates = [root]
    bests = [root]
    while len(candidates)!=0:
        new_candidates = []
        for candidate in candidates:
            new_candidates.extend(candidate['children'])
        if len(new_candidates)>0:    
            if eval_mode == 'model':
                for new_candidate in new_candidates: # sort candidates in both cases (with and without enforcement)
                    new_candidate['prediction'] = preds_dict[new_candidate['schedule']]
                new_candidates = sorted(new_candidates, key = lambda x: x['prediction'],reverse=True)
                if new_candidates[0]['level_has_enforcement']: # if some enforcement is set at this level 
                    candidates= [candidate for candidate in new_candidates if candidate['enforced']] # take the enforced childs only
                else: # if no enforcement, take BS best
                    candidates = new_candidates[:beam_size]
            elif eval_mode == 'execution':
                new_candidates = sorted(new_candidates, key = lambda x: x['evaluation'])
                candidates = new_candidates[:beam_size]

            bests.append(new_candidates[0])
        else:
            candidates= new_candidates #empty list
        
    if eval_mode == 'model':
        return max(bests, key = lambda x: x['prediction'])
    
    elif eval_mode == 'execution':
        return min(bests, key = lambda x: x['evaluation'])
        

def simulate_BeamSearch_on_Dataset(dataset, predictions_df, enforced_scheds_df,true_beam_search=False, get='speedups'):
    # I added the ground truth ^^
    if true_beam_search:
        bs_func = simulate_TrueBeamSearch
    else:
        bs_func = simulate_BeamSearch
    assert get in ['speedups', 'schedules']
        
    df = pd.DataFrame(columns = ['name','nb_scheds','base_time', 'eval_mode']+['bs='+str(i) for i in range(1,10)])
    for func_name in tqdm(sorted(list(predictions_df['name'].unique()),reverse=True)):
        
        func_dict = dataset.programs_dict[func_name]

        nb_scheds = len(func_dict['schedules_list'])
        init_exec_time = func_dict['initial_execution_time']
        root = update_exploration_trace(func_dict['exploration_trace'], predictions_df.query('name==@func_name'), enforced_scheds_df.query('name==@func_name'))
        root['depth'] = 0
#         best_candidate = simulate_BeamSearch(root,9999999,eval_mode='execution')
#         best_sp = round(root['evaluation']/best_candidate['evaluation'],2)
#         best_sched = best_candidate['schedule']
        sp_per_bs = dict()
        predictions_dict = predictions_df.query('name==@func_name')[['sched_str','prediction']].set_index('sched_str').to_dict()['prediction']    
#         print(func_name)
        for eval_mode in ['execution','model', 'level1']:
            sp_per_bs[eval_mode]=[]
            for i in range(1,10): #10 = max beam size
                if get=='schedules':
                    sp_per_bs[eval_mode].append(bs_func(root,i,preds_dict=predictions_dict,eval_mode=eval_mode)['schedule'])
                else:
                    sp_per_bs[eval_mode].append(round(root['evaluation']/bs_func(root,i,preds_dict=predictions_dict,eval_mode=eval_mode)['evaluation'],2)) ###  why is it evaluation ??

        df.loc[len(df)] = [func_name, str(nb_scheds), str(init_exec_time), 'execution'] + [i for i in sp_per_bs['execution']]
        df.loc[len(df)] = [func_name, str(nb_scheds), str(init_exec_time), 'model'] + [i for i in sp_per_bs['model']]
        df.loc[len(df)] = [func_name, str(nb_scheds), str(init_exec_time), 'level1'] + [i for i in sp_per_bs['level1']]
    return df

def get_search_performance(dataset, predictions_df, enforced_scheds_df, true_beam_search=False, tira=True):
    if true_beam_search:
        bs_func = simulate_TrueBeamSearch
    else:
        bs_func = simulate_BeamSearch
        
    df = pd.DataFrame(columns = ['name','nb_scheds','base_time','tira']+['bs='+str(i) for i in range(1,10)])
    for func_name in tqdm(sorted(list(predictions_df['name'].unique()),reverse=True)):
        
        func_dict = dataset.programs_dict[func_name]

        nb_scheds = len(func_dict['schedules_list'])
        init_exec_time = func_dict['initial_execution_time']
        root = update_exploration_trace(func_dict['exploration_trace'], predictions_df.query('name==@func_name'), enforced_scheds_df.query('name==@func_name'))
        root['depth'] = 0
#         best_candidate = simulate_BeamSearch(root,9999999,eval_mode='execution')
#         best_sp = round(root['evaluation']/best_candidate['evaluation'],2)
#         best_sched = best_candidate['schedule']
        sp_per_bs = dict()
        predictions_dict = predictions_df.query('name==@func_name')[['sched_str','prediction']].set_index('sched_str').to_dict()['prediction']    
        for eval_mode in ['execution','model', 'level1']:
            sp_per_bs[eval_mode]=[]
            for i in range(1,10):
                sp_per_bs[eval_mode].append((round(root['evaluation']/bs_func(root,i,preds_dict=predictions_dict,eval_mode=eval_mode)['evaluation'],2)))
                
#         only difference with function with schedules
        if tira==True:
            df.loc[len(df)] = [func_name, str(nb_scheds), str(init_exec_time), 'True'] + [sp_per_bs['model'][i]/sp_per_bs['execution'][i]*100 for i in range(len(sp_per_bs['execution']))]
        else:
            df.loc[len(df)] = [func_name, str(nb_scheds), str(init_exec_time), 'False'] + [sp_per_bs['level1'][i]/sp_per_bs['execution'][i]*100 for i in range(len(sp_per_bs['execution']))]
    return df


#### Functions for Cost model

In [38]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from os import environ
from pprint import pprint
import pickle
import numpy as np
import torch 
import pandas as pd
import seaborn as sns
from torch import optim
import time
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import copy
import random
import plotly.graph_objects as go
import sys
import re
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
import json

# class NbAccessException(Exception):
#     pass
# class LoopsDepthException(Exception):
#     pass

# train_device= torch.device(environ.get('train_device'))
# store_device= torch.device(environ.get('store_device'))

# In[2]:


def mape_criterion(inputs, targets):
    eps = 1e-5
    return 100*torch.mean(torch.abs(targets - inputs)/(targets+eps))
def get_tree_footprint(tree):
    footprint='<L'+str(int(tree['loop_index']))+'>'
    if tree['has_comps']:
        footprint+='['
        for idx in tree['computations_indices']:
            footprint+='C'+str(int(idx))
        footprint+=']'
    for child in tree['child_list']:
        footprint+= get_tree_footprint(child)
    footprint+='</L'+str(int(tree['loop_index']))+'>'
    return footprint
class Model_Recursive_LSTM_v2(nn.Module):
    def __init__(self, input_size, comp_embed_layer_sizes=[600, 350, 200, 180], drops=[0.225, 0.225, 0.225, 0.225], output_size=1):
        super().__init__()
        embedding_size = comp_embed_layer_sizes[-1]
        regression_layer_sizes = [embedding_size] + comp_embed_layer_sizes[-2:]
        concat_layer_sizes = [embedding_size*2+20] + comp_embed_layer_sizes[-2:]
        comp_embed_layer_sizes = [input_size] + comp_embed_layer_sizes
        self.comp_embedding_layers = nn.ModuleList()
        self.comp_embedding_dropouts= nn.ModuleList()
        self.regression_layers = nn.ModuleList()
        self.regression_dropouts= nn.ModuleList()
        self.concat_layers = nn.ModuleList()
        self.concat_dropouts= nn.ModuleList()
        for i in range(len(comp_embed_layer_sizes)-1):
            self.comp_embedding_layers.append(nn.Linear(comp_embed_layer_sizes[i], comp_embed_layer_sizes[i+1], bias=True))
#             nn.init.xavier_uniform_(self.comp_embedding_layers[i].weight)
            nn.init.zeros_(self.comp_embedding_layers[i].weight)
            self.comp_embedding_dropouts.append(nn.Dropout(drops[i]))
        for i in range(len(regression_layer_sizes)-1):
            self.regression_layers.append(nn.Linear(regression_layer_sizes[i], regression_layer_sizes[i+1], bias=True))
#             nn.init.xavier_uniform_(self.regression_layers[i].weight)
            nn.init.zeros_(self.regression_layers[i].weight)
            self.regression_dropouts.append(nn.Dropout(drops[i]))
        for i in range(len(concat_layer_sizes)-1):
            self.concat_layers.append(nn.Linear(concat_layer_sizes[i], concat_layer_sizes[i+1], bias=True))
#             nn.init.xavier_uniform_(self.concat_layers[i].weight)
            nn.init.zeros_(self.concat_layers[i].weight)
            self.concat_dropouts.append(nn.Dropout(drops[i]))
        self.predict = nn.Linear(regression_layer_sizes[-1], output_size, bias=True)
#         nn.init.xavier_uniform_(self.predict.weight)
        nn.init.zeros_(self.predict.weight)
        self.ELU=nn.ELU()
        self.no_comps_tensor = nn.Parameter(nn.init.xavier_uniform_(torch.zeros(1, embedding_size)))
        self.no_nodes_tensor = nn.Parameter(nn.init.xavier_uniform_(torch.zeros(1, embedding_size)))
        self.comps_lstm = nn.LSTM(comp_embed_layer_sizes[-1], embedding_size, batch_first=True)
        self.nodes_lstm = nn.LSTM(comp_embed_layer_sizes[-1], embedding_size, batch_first=True)
        
    def get_hidden_state(self, node, comps_embeddings, loops_tensor):
        nodes_list = []
        for n in node['child_list']:
            nodes_list.append(self.get_hidden_state(n, comps_embeddings,loops_tensor))
        if (nodes_list != []):
            nodes_tensor = torch.cat(nodes_list, 1) 
            lstm_out, (nodes_h_n, nodes_c_n) = self.nodes_lstm(nodes_tensor)
            nodes_h_n = nodes_h_n.permute(1,0,2)
        else:       
            nodes_h_n = torch.unsqueeze(self.no_nodes_tensor, 0).expand(comps_embeddings.shape[0], -1, -1)
        if (node['has_comps']):
            selected_comps_tensor = torch.index_select(comps_embeddings, 1, node['computations_indices'])
            lstm_out, (comps_h_n, comps_c_n) = self.comps_lstm(selected_comps_tensor) 
            comps_h_n = comps_h_n.permute(1,0,2)
        else:
            comps_h_n = torch.unsqueeze(self.no_comps_tensor, 0).expand(comps_embeddings.shape[0], -1, -1)
        selected_loop_tensor = torch.index_select(loops_tensor,1,node['loop_index'])
        x = torch.cat((nodes_h_n, comps_h_n, selected_loop_tensor),2)
        for i in range(len(self.concat_layers)):
            x = self.concat_layers[i](x)
            x = self.concat_dropouts[i](self.ELU(x))
        return x  

    def forward(self, tree_tensors):
        tree, comps_tensor, loops_tensor = tree_tensors
        #computation embbedding layer
        x = comps_tensor
        for i in range(len(self.comp_embedding_layers)):
            x = self.comp_embedding_layers[i](x)
            x = self.comp_embedding_dropouts[i](self.ELU(x))  
        comps_embeddings = x
        #recursive loop embbeding layer
        prog_embedding = self.get_hidden_state(tree, comps_embeddings, loops_tensor)
        #regression layer
        x = prog_embedding
        for i in range(len(self.regression_layers)):
            x = self.regression_layers[i](x)
            x = self.regression_dropouts[i](self.ELU(x))
        out = self.predict(x)
            
        return self.ELU(out[:,0,0])


# In[3]:


def load_data(train_val_dataset_file, split_ratio=None, max_batch_size=2048, drop_sched_func=None, drop_prog_func=None, default_eval=None, speedups_clip_func=None):
    print("loading batches from: "+train_val_dataset_file)
    dataset = Dataset(train_val_dataset_file, max_batch_size, drop_sched_func, drop_prog_func, default_eval, speedups_clip_func)
    if split_ratio == None:
        split_ratio=0.2
    if split_ratio > 1 : # not a ratio a number of batches
        validation_size = split_ratio
    else:
        validation_size = int(split_ratio * len(dataset))
    indices = list(range(len(dataset)))
#     random.Random(42).shuffle(indices)
    val_batches_indices, train_batches_indices = indices[:validation_size],                                               indices[validation_size:]
    val_batches_list=[]
    train_batches_list=[]
    for i in val_batches_indices:
        val_batches_list.append(dataset[i])
    for i in train_batches_indices:
        train_batches_list.append(dataset[i])
    print("Data loaded")
    print("Sizes: "+str((len(val_batches_list),len(train_batches_list)))+" batches")
    return dataset, val_batches_list, val_batches_indices, train_batches_list, train_batches_indices


def get_representation_template(program_dict, max_depth):
    max_accesses = 15
    min_accesses = 1
#     max_depth = 5 
    
    comps_repr_templates_list = []
    comps_indices_dict = dict()
    comps_placeholders_indices_dict = dict()
    
    program_json = program_dict['program_annotation']
    computations_dict = program_json['computations']
    ordered_comp_list = sorted(list(computations_dict.keys()), key = lambda x: computations_dict[x]['absolute_order'])
    
    for comp_index, comp_name in enumerate(ordered_comp_list):
        comp_dict = computations_dict[comp_name]
        if len(comp_dict['accesses'])>max_accesses:
            raise NbAccessException
        if len(comp_dict['accesses'])<min_accesses:
            raise NbAccessException
        if len(comp_dict['iterators'])>max_depth:
            raise LoopsDepthException
            
        comp_repr_template = []
        # Is this computation a reduction 
        comp_repr_template.append(+comp_dict['comp_is_reduction'])


#         iterators representation + tiling and interchage
        iterators_repr = []
        for iter_i,iterator_name in enumerate(comp_dict['iterators']):
            iterator_dict = program_json['iterators'][iterator_name]
            iterators_repr.extend([iterator_dict['lower_bound'], iterator_dict['upper_bound']])
            
            # transformations placeholders
            c_code = 'C'+str(comp_index)
            l_code= c_code+'-L'+str(iter_i)
            iterators_repr.extend([l_code+'Parallelized',
                                   l_code+'Tiled', l_code+'TileFactor',
                                   l_code+'Fused']) #unrolling is skipped since it is only applied on innermost loop

        # Adding padding
        iterator_repr_size = int(len(iterators_repr)/len(comp_dict['iterators']))
        iterators_repr.extend([0]*iterator_repr_size*(max_depth-len(comp_dict['iterators']))) # adding iterators padding 

        # Adding unrolling placeholder since unrolling can only be applied to the innermost loop 
        iterators_repr.extend([c_code+'-Unrolled', c_code+'-UnrollFactor'])
        
        # Adding transformation matrix place holder
        iterators_repr.append(c_code+'-TransformationMatrixStart')
        iterators_repr.extend(['M']*((max_depth+1)**2-2))
        iterators_repr.append(c_code+'-TransformationMatrixEnd')
    
        # Adding the iterators representation to computation vector
        comp_repr_template.extend(iterators_repr)     

        #  Write access representation to computation vector
        padded_write_matrix = pad_access_matrix(isl_to_write_matrix(comp_dict['write_access_relation']), max_depth)
        write_access_repr = [comp_dict['write_buffer_id']+1] + padded_write_matrix.flatten().tolist() # buffer_id + flattened access matrix 
        
        # Adding write access representation to computation vector
        comp_repr_template.extend(write_access_repr)

        # Read Access representation 
        read_accesses_repr=[]
        for read_access_dict in comp_dict['accesses']:
            read_access_matrix = pad_access_matrix(read_access_dict['access_matrix'], max_depth)
            read_access_repr = [+read_access_dict['access_is_reduction']]+ [read_access_dict['buffer_id']+1] + read_access_matrix.flatten().tolist() # buffer_id + flattened access matrix 
            read_accesses_repr.extend(read_access_repr)

        access_repr_len = (max_depth+1)*(max_depth + 2) + 1 +1 # access matrix size +1 for buffer id +1 for is_access_reduction
        read_accesses_repr.extend([0]*access_repr_len*(max_accesses-len(comp_dict['accesses']))) #adding accesses padding

    
        comp_repr_template.extend(read_accesses_repr)

        # Adding Operations count to computation vector
        comp_repr_template.append(comp_dict['number_of_additions'])
        comp_repr_template.append(comp_dict['number_of_subtraction'])
        comp_repr_template.append(comp_dict['number_of_multiplication'])
        comp_repr_template.append(comp_dict['number_of_division'])
        

        # adding log(x+1) of the representation
#         log_rep = list(np.log1p(comp_representation))
#         comp_representation.extend(log_rep)
        
        comps_repr_templates_list.append(comp_repr_template)
        comps_indices_dict[comp_name] = comp_index
        for j, element in enumerate(comp_repr_template):
            if isinstance(element, str):
                comps_placeholders_indices_dict[element] = (comp_index,j)
    

        
    #building loop representation template
    
    loops_repr_templates_list = []
    loops_indices_dict = dict()
    loops_placeholders_indices_dict = dict()
#     assert len(program_json['iterators'])==len(set(program_json['iterators'])) #just to make sure that loop names are not duplicates, but this can't happen because it's a dict
    for loop_index, loop_name in enumerate(program_json['iterators']): # !! is the order in this list fix? can't we get new indices during schedule repr !!! should we use loop name in plchldrs instead of index ? !! #Edit: now it's using the name, so this issue shouldn't occure
        loop_repr_template=[]
        l_code = 'L'+loop_name
        # upper and lower bound
        loop_repr_template.extend([program_json['iterators'][loop_name]['lower_bound'],program_json['iterators'][loop_name]['upper_bound']])   
        loop_repr_template.extend([l_code+'Parallelized',
                                   l_code+'Tiled', l_code+'TileFactor',
                                   l_code+'Fused',
                                   l_code+'Unrolled', l_code+'UnrollFactor'])
        loop_repr_template.extend([l_code+'TransfMatRowStart']+['M']*(max_depth-2+1)+[l_code+'TransfMatRowEnd']) #+1 for the frame
        loop_repr_template.extend([l_code+'TransfMatColStart']+['M']*(max_depth-2+1)+[l_code+'TransfMatColEnd'])
        # adding log(x+1) of the loop representation
        loops_repr_templates_list.append(loop_repr_template)    
        loops_indices_dict[loop_name]=loop_index
        
        for j, element in enumerate(loop_repr_template):
            if isinstance(element, str):
                loops_placeholders_indices_dict[element] = (loop_index,j)
    
            
     
    def update_tree_atributes(node):     
        node['loop_index'] = torch.tensor(loops_indices_dict[node['loop_name']]).to(train_device)
        if node['computations_list']!=[]:
            node['computations_indices'] = torch.tensor([comps_indices_dict[comp_name] for comp_name in node['computations_list']]).to(train_device)
            node['has_comps'] = True
        else:
            node['has_comps'] = False
        for child_node in node['child_list']:
            update_tree_atributes(child_node)
        return node
    
    # getting the original tree structure 
    no_sched_json = program_dict['schedules_list'][0]
    assert 'fusions' not in no_sched_json or no_sched_json['fusions']==None
    orig_tree_structure = no_sched_json['tree_structure']
    tree_annotation = copy.deepcopy(orig_tree_structure) #to avoid altering the original tree from the json
    prog_tree = update_tree_atributes(tree_annotation) 
    
#     loops_tensor = torch.unsqueeze(torch.FloatTensor(loops_repr_templates_list),0)#.to(device)
#     computations_tensor = torch.unsqueeze(torch.FloatTensor(comps_repr_templates_list),0)#.to(device)     

    return prog_tree, comps_repr_templates_list, loops_repr_templates_list, comps_placeholders_indices_dict, loops_placeholders_indices_dict


def get_schedule_representation(program_json, schedule_json, comps_repr_templates_list, loops_repr_templates_list, comps_placeholders_indices_dict, loops_placeholders_indices_dict, max_depth):

    comps_repr = copy.deepcopy(comps_repr_templates_list)
    loops_repr = copy.deepcopy(loops_repr_templates_list)
    
    computations_dict = program_json['computations']
    ordered_comp_list = sorted(list(computations_dict.keys()), key = lambda x: computations_dict[x]['absolute_order'])
    
    padded_tranf_mat_per_comp = dict()
    
    for comp_index, comp_name in enumerate(ordered_comp_list):
        comp_dict =  program_json['computations'][comp_name]
        comp_schedule_dict=schedule_json[comp_name]
        c_code = 'C'+str(comp_index)
        
        
        #Fusion representation
        fused_levels = []
        if 'fusions' in schedule_json and schedule_json['fusions']:
            for fusion in schedule_json['fusions']:#check if comp is involved in fusions 
                 # fusions format [compname1, compname2, loop depth]
                if comp_name in fusion:
                    fused_levels.append(fusion[2])
                
            
        for iter_i,iterator_name in enumerate(comp_dict['iterators']):
            
            ### Updating the computations representation template 
            l_code= c_code+'-L'+str(iter_i)
            
             # Parallelization representation
            parallelized = 0
            if iterator_name == comp_schedule_dict['parallelized_dim']:
                parallelized = 1 # parallelized true
            p_index = comps_placeholders_indices_dict[l_code+'Parallelized']
            comps_repr[p_index[0]][p_index[1]]=parallelized
            
            # Tiling representation 
            tiled = 0
            tile_factor = 0
            if comp_schedule_dict['tiling'] and (iterator_name in comp_schedule_dict['tiling']['tiling_dims']):
                tiled = 1 #tiled: true
                tile_factor_index = comp_schedule_dict['tiling']['tiling_dims'].index(iterator_name)
                tile_factor = int(comp_schedule_dict['tiling']['tiling_factors'][tile_factor_index]) #tile factor
            p_index = comps_placeholders_indices_dict[l_code+'Tiled']
            comps_repr[p_index[0]][p_index[1]] = tiled
            p_index = comps_placeholders_indices_dict[l_code+'TileFactor']
            comps_repr[p_index[0]][p_index[1]] = tile_factor
            
            # Fusion representation
            fused = 0
            if iter_i in fused_levels:
                fused=1
            p_index = comps_placeholders_indices_dict[l_code+'Fused']
            comps_repr[p_index[0]][p_index[1]] = fused
            

         # Unrolling Representation 
        unrolled = 0
        unroll_factor = 0
        if comp_schedule_dict['unrolling_factor']: #Unrolling is always aplied to the innermost loop 
            unrolled=1 #unrolled True
            unroll_factor = int(comp_schedule_dict['unrolling_factor']) #unroll factor
        p_index = comps_placeholders_indices_dict[c_code+'-Unrolled']
        comps_repr[p_index[0]][p_index[1]] = unrolled
        p_index = comps_placeholders_indices_dict[c_code+'-UnrollFactor']
        comps_repr[p_index[0]][p_index[1]] = unroll_factor
        
        # Adding the transformation matrix
        # get the matrix start and end indices 
        mat_start = comps_placeholders_indices_dict[c_code+'-TransformationMatrixStart']
        mat_end = comps_placeholders_indices_dict[c_code+'-TransformationMatrixEnd']
        nb_mat_elements = mat_end[1] - mat_start[1] + 1
        max_depth = int(np.sqrt(nb_mat_elements))-1 # temporarily hack to get max_depth to use it in padding
        padded_matrix = get_padded_transformation_matrix(program_json, schedule_json, comp_name, max_depth)
    #     print(nb_mat_elements, padded_matrix, max_depth)
        assert len(padded_matrix.flatten().tolist()) == nb_mat_elements
    #     print(nb_mat_elements)
        comps_repr[mat_start[0]][mat_start[1]:mat_end[1]+1] = padded_matrix.flatten().tolist() 
        
        padded_tranf_mat_per_comp[comp_name] = padded_matrix #saving it for later to be used in loop repr
        
#     # transforming the schedule_json in order to have loops as key instead of computations, this dict helps building the loop vectors
    loop_schedules_dict = dict()
    for loop_name in program_json['iterators']:
        loop_schedules_dict[loop_name]=dict()
        loop_schedules_dict[loop_name]['TransformationMatrixCol']=[]
        loop_schedules_dict[loop_name]['TransformationMatrixRow']=[]
        loop_schedules_dict[loop_name]['tiled']=0
        loop_schedules_dict[loop_name]['tile_factor']=0
        loop_schedules_dict[loop_name]['unrolled']=0
        loop_schedules_dict[loop_name]['unroll_factor']=0
        loop_schedules_dict[loop_name]['parallelized']=0
        loop_schedules_dict[loop_name]['fused']=0
        
    for comp_index, comp_name in enumerate(ordered_comp_list):
        comp_schedule_dict = schedule_json[comp_name]
        if comp_schedule_dict['tiling']:
            for tiled_loop_index,tiled_loop in enumerate(comp_schedule_dict['tiling']['tiling_dims']):
                loop_schedules_dict[tiled_loop]['tiled']=1
                assert loop_schedules_dict[tiled_loop]['tile_factor']==0 or loop_schedules_dict[tiled_loop]['tile_factor']==int(comp_schedule_dict['tiling']['tiling_factors'][tiled_loop_index]) #just checking that it hasn't been updated with a different value
                loop_schedules_dict[tiled_loop]['tile_factor']=int(comp_schedule_dict['tiling']['tiling_factors'][tiled_loop_index])
        if comp_schedule_dict['unrolling_factor']:
            comp_innermost_loop=computations_dict[comp_name]['iterators'][-1] 
            loop_schedules_dict[comp_innermost_loop]['unrolled']=1
            assert loop_schedules_dict[comp_innermost_loop]['unroll_factor']==0 or loop_schedules_dict[comp_innermost_loop]['unroll_factor']==int(comp_schedule_dict['unrolling_factor'])  #just checking that it hasn't been updated with a different value
            loop_schedules_dict[comp_innermost_loop]['unroll_factor']=int(comp_schedule_dict['unrolling_factor'])
        if comp_schedule_dict['parallelized_dim']:
            loop_schedules_dict[comp_schedule_dict['parallelized_dim']]['parallelized'] = 1
        
        # get the rows and cols transformation matrix for each iterator
        assert padded_tranf_mat_per_comp[comp_name].shape == (max_depth+1,max_depth+1) # make sure that the padding frame is applied, otherwise need to remove the +1 from iter_i+1 in the next few lines 
        for iter_i, loop_name in enumerate(computations_dict[comp_name]['iterators']):
            if len(loop_schedules_dict[loop_name]['TransformationMatrixCol'])>0:#if not empty
                assert (loop_schedules_dict[loop_name]['TransformationMatrixCol'] == padded_tranf_mat_per_comp[comp_name][:,iter_i+1]).all() #chck if the iterator what affected by a different matrix, that shouldn't happen
            else:
                loop_schedules_dict[loop_name]['TransformationMatrixCol'] = padded_tranf_mat_per_comp[comp_name][:,iter_i+1] #+1 for the padding frame
            if len(loop_schedules_dict[loop_name]['TransformationMatrixRow'])>0:#if not empty
                assert (loop_schedules_dict[loop_name]['TransformationMatrixRow'] == padded_tranf_mat_per_comp[comp_name][iter_i+1,:]).all() #chck if the iterator what affected by a different matrix, that shouldn't happen
            else:
                loop_schedules_dict[loop_name]['TransformationMatrixRow'] = padded_tranf_mat_per_comp[comp_name][iter_i+1,:]#+1 for the padding frame
    
    #update the fusions in loops dict 
    if 'fusions' in schedule_json and schedule_json['fusions']:
        for fusion in schedule_json['fusions']:
            fused_loop1 = computations_dict[fusion[0]]['iterators'][fusion[2]]
            fused_loop2 = computations_dict[fusion[1]]['iterators'][fusion[2]]
            loop_schedules_dict[fused_loop1]['fused']=1
            loop_schedules_dict[fused_loop2]['fused']=1
        
# Updating the loop representation templates
    for loop_name in program_json['iterators']:
        l_code = 'L'+loop_name
        
        p_index = loops_placeholders_indices_dict[l_code+'Parallelized']
        loops_repr[p_index[0]][p_index[1]] = loop_schedules_dict[loop_name]['parallelized']
        
        p_index = loops_placeholders_indices_dict[l_code+'Tiled']
        loops_repr[p_index[0]][p_index[1]] = loop_schedules_dict[loop_name]['tiled']
        p_index = loops_placeholders_indices_dict[l_code+'TileFactor']
        loops_repr[p_index[0]][p_index[1]] = loop_schedules_dict[loop_name]['tile_factor']
        
        p_index = loops_placeholders_indices_dict[l_code+'Unrolled']
        loops_repr[p_index[0]][p_index[1]] = loop_schedules_dict[loop_name]['unrolled']
        p_index = loops_placeholders_indices_dict[l_code+'UnrollFactor']
        loops_repr[p_index[0]][p_index[1]] = loop_schedules_dict[loop_name]['unroll_factor']
        
        p_index = loops_placeholders_indices_dict[l_code+'Fused']
        loops_repr[p_index[0]][p_index[1]] = loop_schedules_dict[loop_name]['fused']
        
        row_start = loops_placeholders_indices_dict[l_code+'TransfMatRowStart']
        row_end = loops_placeholders_indices_dict[l_code+'TransfMatRowEnd']
        nb_row_elements = row_end[1] - row_start[1] + 1
        assert len(loop_schedules_dict[loop_name]['TransformationMatrixRow']) == nb_row_elements
        loops_repr[row_start[0]][row_start[1]:row_end[1]+1] = loop_schedules_dict[loop_name]['TransformationMatrixRow']
        
        col_start = loops_placeholders_indices_dict[l_code+'TransfMatColStart']
        col_end = loops_placeholders_indices_dict[l_code+'TransfMatColEnd']
        nb_col_elements = col_end[1] - col_start[1] + 1
        assert len(loop_schedules_dict[loop_name]['TransformationMatrixCol']) == nb_col_elements
        loops_repr[col_start[0]][col_start[1]:col_end[1]+1] = loop_schedules_dict[loop_name]['TransformationMatrixCol']
    
    loops_tensor = torch.unsqueeze(torch.FloatTensor(loops_repr),0)#.to(device)
    computations_tensor = torch.unsqueeze(torch.FloatTensor(comps_repr),0)#.to(device)     

    return computations_tensor, loops_tensor


global_dioph_sols_dict = dict()
def get_padded_transformation_matrix(program_json, schedule_json, comp_name, max_depth=None):
    comp_name = list(program_json['computations'].keys())[0] # for single comp programs, there is only one computation
    comp_dict =  program_json['computations'][comp_name]
    comp_schedule_dict=schedule_json[comp_name]
    nb_iterators = len(comp_dict['iterators'])
    loop_nest = comp_dict['iterators'][:]
    
    if 'transformation_matrix' in comp_schedule_dict: # if the program is explored using matrices
        if comp_schedule_dict['transformation_matrix']!=[]: #if matrix applied, else set it to identity
            assert np.sqrt(len(comp_schedule_dict['transformation_matrix']))==nb_iterators
            final_mat = np.array(list(map(int,comp_schedule_dict['transformation_matrix']))).reshape(nb_iterators,nb_iterators)
        else:
            final_mat = np.zeros((nb_iterators,nb_iterators),int)
            np.fill_diagonal(final_mat,1)
        # just for checking
        comparison_matrix = np.zeros((nb_iterators,nb_iterators),int)
        np.fill_diagonal(comparison_matrix,1)
        for mat in comp_schedule_dict['transformation_matrices'][::-1]:
            comparison_matrix = comparison_matrix@np.array(list(map(int,mat))).reshape(nb_iterators,nb_iterators)
        assert (comparison_matrix==final_mat).all()
    else: # if the program is explored using tags
        interchange_matrix = np.zeros((nb_iterators,nb_iterators),int)
        np.fill_diagonal(interchange_matrix,1)
        if comp_schedule_dict['interchange_dims']:
            first_iter_index = loop_nest.index(comp_schedule_dict['interchange_dims'][0])
            second_iter_index = loop_nest.index(comp_schedule_dict['interchange_dims'][1])
            interchange_matrix[first_iter_index,first_iter_index]=0 #zeroing the diagonal elements
            interchange_matrix[second_iter_index,second_iter_index]=0 #zeroing the diagonal elements
            interchange_matrix[first_iter_index, second_iter_index]=1
            interchange_matrix[second_iter_index, first_iter_index]=1
            loop_nest[first_iter_index], loop_nest[second_iter_index] = loop_nest[second_iter_index], loop_nest[first_iter_index] # swapping iterators in loop nest

        skewing_matrix = np.zeros((nb_iterators,nb_iterators),int)
        np.fill_diagonal(skewing_matrix,1)
        if comp_schedule_dict['skewing']:
            first_iter_index = loop_nest.index(comp_schedule_dict['skewing']['skewed_dims'][0])
            second_iter_index = loop_nest.index(comp_schedule_dict['skewing']['skewed_dims'][1])
            first_factor = int(comp_schedule_dict['skewing']['skewing_factors'][0])
            second_factor = int(comp_schedule_dict['skewing']['skewing_factors'][1])
            # the skewing sub matrix should be in the form of 
            # [[fact1, fact2],
            #  [a,   , b    ]]
            # and we need to find a and b to make to matix det==1
    #         a, b = symbols('a b')
    #         sol = diophantine(first_factor*b - second_factor*a - 1) # solve the diophantine equation to keep a determinant of 1 in the matrix, 
    #         a, b = list(sol)[0] # since we know that there should at least (or only?) one solution 
    #         free_symbol = list(a.free_symbols)[0] # since we know that there should be only one free symbol
    #         a = int(a.subs({free_symbol:0})) #substitue the free symbol with 0 to get the initial solution
    #         b = int(b.subs({free_symbol:0}))
#             sol = simple_linear_diophantine_r(first_factor,second_factor)
            if (first_factor,second_factor) in global_dioph_sols_dict:
                a, b = global_dioph_sols_dict[(first_factor,second_factor)]
            else: 
                a, b = linear_diophantine_default(first_factor,second_factor)
            skewing_matrix[first_iter_index,first_iter_index] = first_factor # update the matrix
            skewing_matrix[first_iter_index,second_iter_index] = second_factor
            skewing_matrix[second_iter_index,first_iter_index] = a
            skewing_matrix[second_iter_index,second_iter_index] = b

        #multiply the mats 
        final_mat = skewing_matrix@interchange_matrix # Right order is skew_mat * interchange_mat
    
    padded_mat = final_mat
    
    
    #pad matrix if max_depth defined
    if max_depth!=None:
        padded_mat = np.c_[np.ones(padded_mat.shape[0]), padded_mat] # adding tags for marking the used rows
        padded_mat = np.r_[[np.ones(padded_mat.shape[1])], padded_mat] # adding tags for marking the used columns
        padded_mat = np.pad(padded_mat, [(0,max_depth-nb_iterators),(0,max_depth-nb_iterators)], mode='constant', constant_values=0)
    
    return padded_mat

    
def get_datapoint_attributes(func_name, program_dict, schedule_index, tree_footprint):
    schedule_json = program_dict['schedules_list'][schedule_index]
    sched_id = str(schedule_index).zfill(4)
    sched_str = sched_json_to_sched_str(schedule_json)
    exec_time = np.min(schedule_json['execution_times'])
    memory_use = program_dict['program_annotation']['memory_size']
    node_name = program_dict['node_name'] if 'node_name' in program_dict else 'unknown'
    speedup = program_dict['initial_execution_time']/exec_time 

    return (func_name, sched_id, sched_str, exec_time, memory_use, node_name, tree_footprint, speedup)

def sched_json_to_sched_str(sched_json): 
    
    if 'sched_str' in sched_json:
        return sched_json['sched_str']
    
    orig_loop_nest = []
    orig_loop_nest.append(sched_json['tree_structure']['loop_name'])
    child_list = sched_json['tree_structure']['child_list']
    while len(child_list)>0:
        child_loop = child_list[0]
        orig_loop_nest.append(child_loop['loop_name'])
        child_list = child_loop['child_list']
        
    comp_name = [n for n in sched_json.keys() if not n in ['unfuse_iterators','tree_structure','execution_times']][0]
    schedule = sched_json[comp_name]
    transf_loop_nest = orig_loop_nest
    sched_str = ''
    
    if 'Transformation Matrix' in schedule:
        if schedule['Transformation Matrix']:
            sched_str+='M('+','.join(schedule['Transformation Matrix'])+')'
    elif "transformation_matrix" in schedule:
        if schedule['transformation_matrix']:
            sched_str+='M('+','.join(schedule['transformation_matrix'])+')'
    if schedule['interchange_dims']:
        first_dim_index = transf_loop_nest.index(schedule['interchange_dims'][0])
        second_dim_index = transf_loop_nest.index(schedule['interchange_dims'][1])
        sched_str+='I(L'+str(first_dim_index)+',L'+str(second_dim_index)+')'
        transf_loop_nest[first_dim_index], transf_loop_nest[second_dim_index] = transf_loop_nest[second_dim_index], transf_loop_nest[first_dim_index]
    if schedule['skewing']:
        first_dim_index = transf_loop_nest.index(schedule['skewing']['skewed_dims'][0])
        second_dim_index = transf_loop_nest.index(schedule['skewing']['skewed_dims'][1])
        first_factor = schedule['skewing']['skewing_factors'][0]
        second_factor = schedule['skewing']['skewing_factors'][1]
        sched_str+='S(L'+str(first_dim_index)+',L'+str(second_dim_index)+','+str(first_factor)+','+str(second_factor)+')'
    if schedule['parallelized_dim']:
        dim_index = transf_loop_nest.index(schedule['parallelized_dim'])
        sched_str+='P(L'+str(dim_index)+')'
    if schedule['tiling']:
        if schedule['tiling']['tiling_depth']==2:
            first_dim = schedule['tiling']['tiling_dims'][0]
            second_dim = schedule['tiling']['tiling_dims'][1]
            first_dim_index = transf_loop_nest.index(first_dim)
            second_dim_index = transf_loop_nest.index(second_dim)
            first_factor = schedule['tiling']['tiling_factors'][0]
            second_factor = schedule['tiling']['tiling_factors'][1]
            sched_str+='T2(L'+str(first_dim_index)+',L'+str(second_dim_index)+','+str(first_factor)+','+str(second_factor)+')'
            i = transf_loop_nest.index(first_dim)
            transf_loop_nest[i:i+1]=first_dim+'_outer', second_dim+'_outer'
            i = transf_loop_nest.index(second_dim)
            transf_loop_nest[i:i+1]=first_dim+'_inner', second_dim+'_inner'
        else: #tiling depth == 3
            first_dim = schedule['tiling']['tiling_dims'][0]
            second_dim = schedule['tiling']['tiling_dims'][1]
            third_dim = schedule['tiling']['tiling_dims'][2]
            first_dim_index = transf_loop_nest.index(first_dim)
            second_dim_index = transf_loop_nest.index(second_dim)
            third_dim_index = transf_loop_nest.index(third_dim)
            first_factor = schedule['tiling']['tiling_factors'][0]
            second_factor = schedule['tiling']['tiling_factors'][1]
            third_factor = schedule['tiling']['tiling_factors'][2]
            sched_str+='T3(L'+str(first_dim_index)+',L'+str(second_dim_index)+',L'+str(third_dim_index)+','+str(first_factor)+','+str(second_factor)+','+str(third_factor)+')'
            i = transf_loop_nest.index(first_dim)
            transf_loop_nest[i:i+1]=first_dim+'_outer', second_dim+'_outer', third_dim+'_outer'
            i = transf_loop_nest.index(second_dim)
            transf_loop_nest[i:i+1]=first_dim+'_inner', second_dim+'_inner', third_dim+'_inner'
            transf_loop_nest.remove(third_dim)
    if schedule['unrolling_factor']:
        dim_index = len(transf_loop_nest)-1
        dim_name =transf_loop_nest[-1]
        sched_str+='U(L'+str(dim_index)+','+schedule['unrolling_factor']+')'
        transf_loop_nest[dim_index:dim_index+1] = dim_name+'_Uouter', dim_name+'_Uinner'
    
    
    return sched_str
    
def pad_access_matrix(access_matrix, max_depth):
    access_matrix = np.array(access_matrix)
    access_matrix = np.c_[np.ones(access_matrix.shape[0]), access_matrix] # adding tags for marking the used rows
    access_matrix = np.r_[[np.ones(access_matrix.shape[1])], access_matrix] # adding tags for marking the used columns
    padded_access_matrix = np.zeros((max_depth + 1, max_depth + 2))
    padded_access_matrix[:access_matrix.shape[0],:access_matrix.shape[1]-1] = access_matrix[:,:-1] #adding padding to the access matrix before the last column
    padded_access_matrix[:access_matrix.shape[0],-1] = access_matrix[:,-1] #appending the last columns
    
    return padded_access_matrix

def isl_to_write_matrix(isl_map): # for now this function only support reductions
    comp_iterators_str = re.findall(r'\[(.*)\]\s*->', isl_map)[0]
    buffer_iterators_str = re.findall(r'->\s*\w*\[(.*)\]', isl_map)[0]
    buffer_iterators_str=re.sub(r"\w+'\s=","",buffer_iterators_str)
    comp_iter_names = re.findall(r'(?:\s*(\w+))+', comp_iterators_str)
    buf_iter_names = re.findall(r'(?:\s*(\w+))+', buffer_iterators_str)
    matrix = np.zeros([len(buf_iter_names),len(comp_iter_names)+1])
    for i,buf_iter in enumerate(buf_iter_names):
        for j,comp_iter in enumerate(comp_iter_names):
            if buf_iter==comp_iter:
                matrix[i,j]=1
                break
    return matrix

def isl_to_write_dims(isl_map): # return the buffer iterator that defines the write buffer
    buffer_iterators_str = re.findall(r'->\s*\w*\[(.*)\]', isl_map)[0]
    buffer_iterators_str = re.sub(r"\w+'\s=","",buffer_iterators_str)
    buf_iter_names = re.findall(r'(?:\s*(\w+))+', buffer_iterators_str)
    return buf_iter_names

def get_results_df(dataset, batches_list, indices, model, log=False):   
    df = pd.DataFrame()
    model.eval()
    torch.set_grad_enabled(False)
    all_outputs=[]
    all_labels=[]
    prog_names=[]
    sched_names=[]
    exec_times=[]
    sched_strs=[]
    memory_uses=[]
    node_names=[]
    tree_footprints = []

    for k, (inputs, labels) in tqdm(list(enumerate(batches_list))):
        original_device = labels.device
        inputs=(inputs[0], inputs[1].to(train_device), inputs[2].to(train_device))
        labels=labels.to(train_device)
        outputs = model(inputs)
        assert outputs.shape == labels.shape
        all_outputs.append(outputs)
        all_labels.append(labels)
#         assert len(outputs)==len(dataset.batched_schedule_names[indices[k]])
#         assert len(outputs)==len(dataset.batched_program_names[indices[k]])
#         for j, sched_name in enumerate(dataset.batched_schedule_names[indices[k]]):
#             sched_names.append(sched_name)
#             prog_names.append(dataset.batched_program_names[indices[k]][j])
#             exec_times.append(dataset.batched_exec_time[indices[k]][j])
        assert len(outputs)==len(dataset.batched_datapoint_attributes[indices[k]])
        zipped_attributes = list(zip(*dataset.batched_datapoint_attributes[indices[k]]))
        prog_names.extend(zipped_attributes[0])
        sched_names.extend(zipped_attributes[1])
        sched_strs.extend(zipped_attributes[2])
        exec_times.extend(zipped_attributes[3])
        memory_uses.extend(zipped_attributes[4])
        node_names.extend(zipped_attributes[5])
        tree_footprints.extend(zipped_attributes[6])
        inputs=(inputs[0], inputs[1].to(original_device), inputs[2].to(original_device))
        labels=labels.to(original_device)
    preds = torch.cat(all_outputs)
    targets = torch.cat(all_labels)
    preds = preds.cpu().detach().numpy().reshape((-1,))
    preds = np.around(preds,decimals=6)
    targets = np.around(targets.cpu().detach().numpy().reshape((-1,)),decimals=6)
                                            
    assert preds.shape == targets.shape 
    df['name'] = prog_names
    df['tree_struct'] = tree_footprints
    df['sched_name'] = sched_names
    df['sched_str'] = sched_strs
    df['exec_time'] = exec_times
    df['memory_use'] = list(map(float,memory_uses))
    df['node_name'] = node_names
    df['prediction'] = np.array(preds)
    df['target'] = np.array(targets)
#     df['abs_diff'] = np.abs(preds - targets)
    df['APE'] = np.abs(df.target - df.prediction)/df.target * 100
    df['sched_str'] = df['sched_str'].apply(lambda x: simplify_sched_str(x))
        
    return df

def simplify_sched_str(sched_str): #checks if the the same matrix is applied multiple computations, then merge the M() parts into a single 
#     print('before ')
    if sched_str.count('M')==1:
        return sched_str
    comps = re.findall('C\d+', sched_str)
    comps = set(comps)
    
    mats = set(re.findall(r'M\({[\dC\,]+},([\d\,\-]+)',sched_str))
    comps_per_mat = {mat:[] for mat in mats}
    new_mats_str = ''
    for mat in comps_per_mat:
        for mat_part in re.findall('M\({[C\d\,]+},'+mat,sched_str):
            comps_per_mat[mat].extend(re.findall('C\d+',mat_part))
        new_mats_str+='M({'+','.join(sorted(comps_per_mat[mat]))+'},'+mat+')'
    return re.sub('(M\({[\dC\,]+},[\d\,\-]+\))+',new_mats_str,sched_str)


# In[5]:


class Dataset():
    def __init__(self, dataset_filename, max_batch_size, drop_sched_func=None, drop_prog_func=None, can_set_default_eval=None , speedups_clip_func=None):
        
        if dataset_filename.endswith('json'):
            with open(dataset_filename, 'r') as f:
                dataset_str= f.read()
            self.programs_dict=json.loads(dataset_str)
        elif dataset_filename.endswith('pkl'):
            with open(dataset_filename, 'rb') as f:
                self.programs_dict = pickle.load(f)
        
        self.batched_X = []
        self.batched_Y = []
        self.batches_dict=dict()
        self.max_depth = 5 #WAS 5, LINA CHANGED IT FOR TESTING MISSING FUNCTIONS
        self.nb_dropped = 0
        self.pgme = 0
        self.nb_pruned = 0
        self.dropped_funcs = []
        self.batched_datapoint_attributes = []
        self.nb_datapoints = 0

        if (drop_sched_func==None):
            drop_sched_func = lambda x,y : False
        if (drop_prog_func==None):
            drop_prog_func = lambda x : False
        if (speedups_clip_func==None):
            speedups_clip_func = lambda x : x
        if (can_set_default_eval==None):
            can_set_default_eval = lambda x,y:0
                
        functions_list = list(self.programs_dict.keys())
        random.Random(42).shuffle(functions_list)
        for function_name in tqdm(functions_list):
            if drop_prog_func(self.programs_dict[function_name]):
                self.nb_dropped += len(self.programs_dict[function_name]['schedules_list'])
                self.pgme += 1
                self.dropped_funcs.append(function_name)
                continue
                
            program_json = self.programs_dict[function_name]['program_annotation']
            
            try:
                prog_tree, comps_repr_templates_list, loops_repr_templates_list, comps_placeholders_indices_dict, loops_placeholders_indices_dict = get_representation_template(self.programs_dict[function_name], max_depth = self.max_depth)
            except (NbAccessException, LoopsDepthException):
                self.nb_dropped += len(self.programs_dict[function_name]['schedules_list'])
                self.pgme += 1
                continue
            program_exec_time = self.programs_dict[function_name]['initial_execution_time']
            tree_footprint=get_tree_footprint(prog_tree)
            self.batches_dict[tree_footprint] = self.batches_dict.get(tree_footprint,{'tree':prog_tree,'comps_tensor_list':[],'loops_tensor_list':[],'datapoint_attributes_list':[],'speedups_list':[],'exec_time_list':[]})
            
            for schedule_index in range(len(self.programs_dict[function_name]['schedules_list'])):
                schedule_json = self.programs_dict[function_name]['schedules_list'][schedule_index]
                sched_exec_time = np.min(schedule_json['execution_times'])
                if drop_sched_func(self.programs_dict[function_name], schedule_index) or (not sched_exec_time):
                    self.nb_dropped +=1
                    self.nb_pruned +=1
                    continue
                
                sched_speedup = program_exec_time / sched_exec_time
                
                def_sp = can_set_default_eval(self.programs_dict[function_name], schedule_index)
                if def_sp>0:
                    sched_speedup = def_sp
                    
                sched_speedup = speedups_clip_func(sched_speedup)

                comps_tensor, loops_tensor = get_schedule_representation(program_json, schedule_json, comps_repr_templates_list, loops_repr_templates_list, comps_placeholders_indices_dict, loops_placeholders_indices_dict, self.max_depth)
                
                datapoint_attributes = get_datapoint_attributes(function_name, self.programs_dict[function_name], schedule_index, tree_footprint)
                
                self.batches_dict[tree_footprint]['comps_tensor_list'].append(comps_tensor)
                self.batches_dict[tree_footprint]['loops_tensor_list'].append(loops_tensor)
                self.batches_dict[tree_footprint]['datapoint_attributes_list'].append(datapoint_attributes)
                self.batches_dict[tree_footprint]['speedups_list'].append(sched_speedup)
                self.nb_datapoints+=1

        storing_device = store_device
        print('Batching ...', self.pgme)
        for tree_footprint in tqdm(self.batches_dict):
            
            #shuffling the lists inside each footprint to avoid having batches with very low program diversity
            zipped = list(zip(self.batches_dict[tree_footprint]['datapoint_attributes_list'],
                              self.batches_dict[tree_footprint]['comps_tensor_list'],
                              self.batches_dict[tree_footprint]['loops_tensor_list'],
                              self.batches_dict[tree_footprint]['speedups_list']))
            random.shuffle(zipped)
            self.batches_dict[tree_footprint]['datapoint_attributes_list'],self.batches_dict[tree_footprint]['comps_tensor_list'],self.batches_dict[tree_footprint]['loops_tensor_list'],self.batches_dict[tree_footprint]['speedups_list']=zip(*zipped)
            
            for chunk in range(0,len(self.batches_dict[tree_footprint]['speedups_list']),max_batch_size):
                if (storing_device.type=='cuda' and (torch.cuda.memory_allocated(storing_device.index)/torch.cuda.get_device_properties(storing_device.index).total_memory)>0.80):  # Check GPU memory in order to avoid Out of memory error
                    print('GPU memory on '+str(storing_device)+' nearly full, switching to CPU memory')
                    storing_device = torch.device('cpu')
                self.batched_datapoint_attributes.append(self.batches_dict[tree_footprint]['datapoint_attributes_list'][chunk:chunk+max_batch_size])
                self.batched_X.append((self.batches_dict[tree_footprint]['tree'],
                               torch.cat(self.batches_dict[tree_footprint]['comps_tensor_list'][chunk:chunk+max_batch_size], 0).to(storing_device),
                               torch.cat(self.batches_dict[tree_footprint]['loops_tensor_list'][chunk:chunk+max_batch_size], 0).to(storing_device)))
                self.batched_Y.append(torch.FloatTensor(self.batches_dict[tree_footprint]['speedups_list'][chunk:chunk+max_batch_size]).to(storing_device))
        
        #shuffling batches to avoid having the same footprint in consecutive batches
        zipped = list(zip(self.batched_X, self.batched_Y, self.batched_datapoint_attributes))
        random.shuffle(zipped)
        self.batched_X, self.batched_Y, self.batched_datapoint_attributes = zip(*zipped)
        
        print(f'Number of datapoints {self.nb_datapoints} Number of batches {len(self.batched_Y)}')
#         del self.programs_dict
        del self.batches_dict
        
    def __getitem__(self, index):
        if isinstance(index, slice):
            start, stop, step = index.indices(len(self))
            return [self[i] for i in range(start, stop, step)]
        elif isinstance(index, int):
            return self.batched_X[index], self.batched_Y[index] 

    def __len__(self):
        return len(self.batched_Y)


# In[6]:


def has_skippable_loop_1comp(prog_dict): # check if the program has a non time-step free iterator 
                                   # (has an iterator that is not used in accesses and the expression doesn't have reduction stentcils)
    
    program_json =  prog_dict['program_annotation']
    if not len(program_json['computations'])==1: #this function works only on single comp programs
        return False
    comp_name = list(program_json['computations'].keys())[0]
    comp_dict = program_json['computations'][comp_name]
    write_buffer_id = comp_dict['write_buffer_id']
    iterators = comp_dict['iterators']
    write_dims =  isl_to_write_dims(comp_dict['write_access_relation'])
    read_buffer_ids = [e['buffer_id'] for e in comp_dict['accesses']]
    
    
    if len(write_dims)==len(iterators): # if all loops used in write, no free loops
        # one special case of empty program
        if len(read_buffer_ids) == 1 and read_buffer_ids[0]==write_buffer_id and comp_dict['number_of_additions'] ==0 and comp_dict['number_of_subtraction'] ==0 and comp_dict['number_of_multiplication'] ==0 and comp_dict['number_of_division'] ==0: 
            return True
        return False
    
    if not write_buffer_id in read_buffer_ids: # if the calculation is clearly overwritten
        return True
    
    # find the simle reduction access
    found = False
    for access in comp_dict['accesses']:
        if access['buffer_id']==write_buffer_id and not access_is_stencil(access):
            found = True
            break
    if not found: # no simple reduction access is found, but we know that there is a reduction access in expression, so there is a skippable loop if the reduction is performed on last iterator, otherwise it's hardly skippable
        if write_dims[-1]!=iterators[-1]: # reduction is performed on the last iterator
            return True
    
    # find the non simple reduction accesses
    for access in comp_dict['accesses']:
        if access['buffer_id']==write_buffer_id and access_is_stencil(access): # a stencil access pattern is used
            return False
    
    # checking if there is a free loop (not used in write nor in read)
    read_dims_bools = []
    for access in comp_dict['accesses']: 
        read_dims_bools.append(np.any(access['access_matrix'], axis=0))
    read_dims_bools = np.any(read_dims_bools,axis=0)
    read_iterators = [iterators[i] for i, is_used in enumerate(read_dims_bools[:-1]) if is_used==True]
    used_iterators = set(write_dims+read_iterators)
    if len(used_iterators)==len(iterators): # all iterators are used in the computation
        return False
    
    if iterators[-1] in used_iterators: # the last iterator is not the dropped one, so the dropped loop shouldn't be skippable (knowing that there is a reduction access)
        if len(comp_dict['accesses'])>2:# has to have more than 2 accesses to make sure the loop isn't skippable, adding this condition for strictness
            return False
        
    return True

def sched_is_prunable_1comp(schedule_str, prog_depth):
    if re.search('P\(L2\)U\(L3,\d+\)', schedule_str):
        return True
    if prog_depth==2:
        if re.search('P\(L1\)(?:[^T]|$)', schedule_str):
            return True
    if prog_depth==3:
        if re.search('P\(L2\)(?:[^T]|$|T2\(L0,L1)', schedule_str):
            return True
    return False

def can_set_default_eval_1comp(schedule_str, prog_depth):
    def_sp = 0
#     print(schedule_str, type(schedule_str))
    if prog_depth==2:
        if re.search('P\(L1\)$', schedule_str):
            def_sp = 0.001
    if prog_depth==3:
        if re.search('P\(L2\)$', schedule_str):
            def_sp = 0.001
    return def_sp
def access_is_stencil(access):
    return np.any(access['access_matrix'], axis=0)[-1]
def linear_diophantine_default(f_i,f_j):
    found = False
    gamma = 0
    sigma = 1
    if ((f_j == 1) or (f_i == 1)):
        gamma = f_i - 1
        sigma = 1
    else:
        if((f_j == -1) and (f_i > 1)):
            gamma = 1
            sigma = 0       
        else:     
            i =0
            while((i < 100) and (not found)):     
                if (((sigma * f_i ) % abs(f_j)) ==  1):
                            found = True
                else:
                    sigma+=1
                    i+=1
            if(not found):
                print('Error cannof find solution to diophantine equation')
                return
            gamma = ((sigma * f_i) - 1 ) / f_j
    
    return gamma, sigma


# In[7]:


def wrongly_pruned_schedule(prog_dict, schedule_index):
    schedule_dict = prog_dict['schedules_list'][schedule_index]
    if not "sched_str" in schedule_dict: # this function concerns multicomp progs only, if sched str not in annot it means that the prog is single comp
        return False 
    sched_str = schedule_dict["sched_str"]
#     if(schedule_dict["execution_times"] == None):
#         return True
    target = prog_dict['initial_execution_time'] / np.min(schedule_dict["execution_times"])
    depths = []
    for depth in prog_dict['program_annotation']['computations']:
        depths.append(len(prog_dict['program_annotation']['computations'][depth]['iterators']))
    if(not (target>0.0113 or target<0.0087)):
        reg_str = ""
        for j in reversed(range(len(depths))):
            for i in range(depths[j]-1):
                reg_str += ".*P\(\{(C[0-9],)*C" + str(j) + "(,C[0-9])*\},L"+ str(i) +"\)$|"
        reg_str= reg_str[:-1]
        if(re.search(reg_str, sched_str)):
#             if len(schedule_dict["execution_times"])==1:
            print(prog_dict['filename'][2:16], schedule_index, sched_str, len(schedule_dict["execution_times"]),'yes')
            
#                             if not 'function760518'<prog_dict['filename'][2:16]<'function761289':
#                 print(prog_dict['filename'][2:16], schedule_index, sched_str, len(schedule_dict["execution_times"]),'are you sure?')
#             else:
#                 print(prog_dict['filename'][2:16], schedule_index, sched_str, len(schedule_dict["execution_times"]),'yes')
#             assert 'function760518'<prog_dict['filename'][2:16]<'function761289' #since we know that the ranges of programs affected by this bug, we can make this assertion
            return True
        else:
            return False
    else:
        return False
    
    
def encode_df_interchage(row, pred = True): #encode the output vector to get '1' in the right LI, '0' elsewhere
    if pred == True:
        str_sched = row["pred_str"]
    else:
        str_sched = row["target_str"]
    output = np.zeros(106,dtype=int) # 106 = 2 C 15 + 1
    str_interchange = re.findall('I\(.*\)', str_sched)  # get the LI string
    if str_interchange==[]:
        output[0] = 1
    else:
        m = re.findall(r'\d+', str_interchange[0])  # get the number of the loops
        a = min(int(m[-2]),int(m[-1]))
        b = max(int(m[-2]),int(m[-1]))
        i2 = b-a
        i1 = pos(a,15)
        output[i1+i2]=1      
    return output

def exist_merged(row, names_merged): #encode the output vector to get '1' in the right LI, '0' elsewhere
    return (row["name"] in names_merged)

def pos(a,depth):  # calculate the position of the interchange in the output vector
    if(a==0):
        return 0
    elif (a==1):
        return depth-1
    else:
        return pos(a-1,depth)+depth-a

#### Functions for LI k-best specialized model

In [17]:
def get_representation_LI(program_json, schedule_json):
    max_dims= 8
    max_accesses = 15 # TODO: check if 10 is enough
    program_representation = []
    indices_dict = dict()
    computations_dict = program_json['computations']
    ordered_comp_list = sorted(list(computations_dict.keys()), key = lambda x: computations_dict[x]['absolute_order'])
    
    for index, comp_name in enumerate(ordered_comp_list):
        comp_dict = computations_dict[comp_name]
        if len(comp_dict['accesses'])>max_accesses:
#             print('too much acc')
            raise LargeAccessMatices
        if len(comp_dict['accesses'])<1:
#             print('too little acc')
            raise LargeAccessMatices
        comp_representation = []
        #         Is this computation a reduction 
        comp_representation.append(+comp_dict['comp_is_reduction'])


#         iterators representation + tiling and interchage
        iterators_repr = []
        for iterator_name in comp_dict['iterators']:
            
            iterator_dict = program_json['iterators'][iterator_name]
            iterators_repr.append(iterator_dict['upper_bound']) 
#             iterators_repr.append(iterator_dict['lower_bound'])
            # unfuse schedule replacing the lower bound for compability issue, this enables the use transfer learning from an older model 
            parent_iterator = program_json['iterators'][iterator_name]['parent_iterator']
            
            # changed bcs it cause an error, to verify  ( as well as line 228) !!!!!!
            #if parent_iterator in schedule_json['unfuse_iterators']:
                #iterators_repr.append(1) #unfused true
            #else:
                #iterators_repr.append(0) #unfused false
            
            
            if iterator_name in schedule_json[comp_name]['interchange_dims']:
                iterators_repr.append(1) #interchanged true
            else:
                iterators_repr.append(0) #interchanged false
            
            # Skewing representation
            skewed = 0
            skew_factor = 0
            skew_extent = 0
            if schedule_json[comp_name]['skewing'] and (iterator_name in schedule_json[comp_name]['skewing']['skewed_dims']):
                skewed = 1 #skewed: true
                skew_factor_index = schedule_json[comp_name]['skewing']['skewed_dims'].index(iterator_name)
                skew_factor = int(schedule_json[comp_name]['skewing']['skewing_factors'][skew_factor_index]) # skew factor
                skew_extent = int(schedule_json[comp_name]['skewing']['average_skewed_extents'][skew_factor_index]) # skew extent
            iterators_repr.append(skewed)
            iterators_repr.append(skew_factor)
            iterators_repr.append(skew_extent)
            
             # Parallelization representation
            parallelized = 0
            if iterator_name == schedule_json[comp_name]['parallelized_dim']:
                parallelized = 1 # parallelized true
            iterators_repr.append(parallelized)
            
            if (schedule_json[comp_name]['tiling']!={}):
                if iterator_name in schedule_json[comp_name]['tiling']['tiling_dims']:
                    iterators_repr.append(1) #tiled: true
                    tile_factor_index = schedule_json[comp_name]['tiling']['tiling_dims'].index(iterator_name)
                    iterators_repr.append(int(schedule_json[comp_name]['tiling']['tiling_factors'][tile_factor_index])) #tile factor
                else:
                    iterators_repr.append(0) #tiled: false
                    iterators_repr.append(0) #tile factor 0
            else: #tiling = None
                iterators_repr.append(0) #tiled: false
                iterators_repr.append(0) #tile factor 0    
            # is this dimension saved (this dimension does not disapear aftre reduction)
#             iterators_repr.append(+(iterator_name in comp_dict['real_dimensions']))
                    
        iterator_repr_size = int(len(iterators_repr)/len(comp_dict['iterators']))
        iterators_repr.extend([0]*iterator_repr_size*(max_dims-len(comp_dict['iterators']))) # adding iterators padding 

        comp_representation.extend(iterators_repr) #adding the iterators representation    

        #       write access represation 
        write_access_matrix = isl_to_write_matrix(comp_dict['write_access_relation'])
        write_access_matrix = np.array(write_access_matrix)
        write_access_matrix = np.c_[np.ones(write_access_matrix.shape[0]), write_access_matrix] # adding tags for marking the used rows
        write_access_matrix = np.r_[[np.ones(write_access_matrix.shape[1])], write_access_matrix] # adding tags for marking the used columns
        padded_write_matrix = np.zeros((max_dims + 1, max_dims + 2))
        padded_write_matrix[:write_access_matrix.shape[0],:write_access_matrix.shape[1]-1] = write_access_matrix[:,:-1] #adding padding to the access matrix
        padded_write_matrix[:write_access_matrix.shape[0],-1] = write_access_matrix[:,-1] #adding padding to the access matrix
        write_access_repr = [comp_dict['write_buffer_id']+1] + padded_write_matrix.flatten().tolist()
        comp_representation.extend(write_access_repr)
        
#         accesses representation
        accesses_repr=[]
        for access_dict in comp_dict['accesses']:
            access_matrix = access_dict['access_matrix']
            access_matrix = np.array(access_matrix)
            padded_access_matrix = np.zeros((max_dims, max_dims + 1))
            padded_access_matrix[:access_matrix.shape[0],:access_matrix.shape[1]-1] = access_matrix[:,:-1] #adding padding to the access matrix
            padded_access_matrix[:access_matrix.shape[0],-1] = access_matrix[:,-1] #adding padding to the access matrix
            access_repr = [access_dict['buffer_id']] + padded_access_matrix.flatten().tolist() # input_id + flattened access matrix 
            # is this access a reduction (the computation is accesing itself)
            access_repr.append(+access_dict['access_is_reduction'])
            accesses_repr.extend(access_repr)

        #access_repr_len = max_dims*(max_dims + 1)
        access_repr_len = max_dims*(max_dims + 1) + 1 +1 #+1 for input id, +1 for is_access_reduction
        accesses_repr.extend([0]*access_repr_len*(max_accesses-len(comp_dict['accesses']))) #adding accesses padding
    
        comp_representation.extend(accesses_repr) #adding access representation

#         operation histogram
        comp_representation.append(comp_dict['number_of_additions'])
        comp_representation.append(comp_dict['number_of_subtraction'])
        comp_representation.append(comp_dict['number_of_multiplication'])
        comp_representation.append(comp_dict['number_of_division'])

        
#         unrolling representation
        if (schedule_json[comp_name]['unrolling_factor']!=None):
            comp_representation.append(1) #unrolled True
            comp_representation.append(int(schedule_json[comp_name]['unrolling_factor'])) #unroll factor
        else:
            comp_representation.append(0) #unrolled false
            comp_representation.append(0) #unroll factor 0

        # adding log(x+1) of the representation
#         log_rep = list(np.log1p(comp_representation))
#         comp_representation.extend(log_rep)
        
        program_representation.append(comp_representation)
        indices_dict[comp_name] = index
    
    # transforming the schedule_json inorder to have loops as key instead of computations, this dict helps building the loop vectors
    loop_schedules_dict = dict()
    for loop_name in program_json['iterators']:
        loop_schedules_dict[loop_name]=dict()
        loop_schedules_dict[loop_name]['interchanged']=False
        loop_schedules_dict[loop_name]['interchanged_with']=None
        loop_schedules_dict[loop_name]['skewed']=False
        loop_schedules_dict[loop_name]['skewed_dims']=None
        loop_schedules_dict[loop_name]['skew_factor']=None
        loop_schedules_dict[loop_name]['skew_extent']=None
        loop_schedules_dict[loop_name]['parallelized']=False
        loop_schedules_dict[loop_name]['tiled']=False
        loop_schedules_dict[loop_name]['tile_depth']=None
        loop_schedules_dict[loop_name]['tiled_dims']=None
        loop_schedules_dict[loop_name]['tile_factor']=None
        loop_schedules_dict[loop_name]['unrolled']=False
        loop_schedules_dict[loop_name]['unroll_factor']=None
        loop_schedules_dict[loop_name]['unroll_comp']=None
        loop_schedules_dict[loop_name]['unfused']=False     
    for comp_name in schedule_json:
        if not comp_name.startswith('comp'): 
            continue # skip the non computation keys
        if schedule_json[comp_name]['interchange_dims']!=[]:
            interchanged_loop1=schedule_json[comp_name]['interchange_dims'][0]
            interchanged_loop2=schedule_json[comp_name]['interchange_dims'][1]
            loop_schedules_dict[interchanged_loop1]['interchanged']=True
            loop_schedules_dict[interchanged_loop1]['interchanged_with']=interchanged_loop2
            loop_schedules_dict[interchanged_loop2]['interchanged']=True
            loop_schedules_dict[interchanged_loop2]['interchanged_with']=interchanged_loop1
        if schedule_json[comp_name]['skewing']:
            for skewed_loop_index,skewed_loop in enumerate(schedule_json[comp_name]['skewing']['skewed_dims']):
                loop_schedules_dict[skewed_loop]['skewed']=True
                loop_schedules_dict[skewed_loop]['skew_factor'] = int(schedule_json[comp_name]['skewing']['skewing_factors'][skewed_loop_index])
                loop_schedules_dict[skewed_loop]['skew_extent'] = int(schedule_json[comp_name]['skewing']['average_skewed_extents'][skewed_loop_index])
        if schedule_json[comp_name]['parallelized_dim']:
             loop_schedules_dict[schedule_json[comp_name]['parallelized_dim']]['parallelized']=True
        if schedule_json[comp_name]['tiling']!={}:
            for tiled_loop_index,tiled_loop in enumerate(schedule_json[comp_name]['tiling']['tiling_dims']):
                loop_schedules_dict[tiled_loop]['tiled']=True
                loop_schedules_dict[tiled_loop]['tile_depth']=schedule_json[comp_name]['tiling']['tiling_depth']
                loop_schedules_dict[tiled_loop]['tiled_dims']=schedule_json[comp_name]['tiling']['tiling_dims']
                loop_schedules_dict[tiled_loop]['tile_factor']=int(schedule_json[comp_name]['tiling']['tiling_factors'][tiled_loop_index])
        if schedule_json[comp_name]['unrolling_factor']!=None:
            comp_innermost_loop=computations_dict[comp_name]['iterators'][-1] 
            tiling_dims = [] if schedule_json[comp_name]['tiling']=={} else schedule_json[comp_name]['tiling']['tiling_dims']
            interchange_dims =schedule_json[comp_name]['interchange_dims']
            if (not ((comp_innermost_loop in tiling_dims)or(comp_innermost_loop in interchange_dims))):#unrolling always applied to innermost loop, if tilling or interchange is applied to innermost, unroll is applied to the resulting loop instead of the orginal, hence we don't represent it
                loop_schedules_dict[comp_innermost_loop]['unrolled']=True
                loop_schedules_dict[comp_innermost_loop]['unroll_factor']=int(schedule_json[comp_name]['unrolling_factor'])
                loop_schedules_dict[comp_innermost_loop]['unroll_comp']=comp_name
    
    #for unfuse_parent in schedule_json['unfuse_iterators'] :
        #for unfused_loop in program_json['iterators'][unfuse_parent]['child_iterators']:
            #loop_schedules_dict[unfused_loop]['unfused']=True
    
    # collect the set of iterators that are used for computation (to eleminate those that are only used for inputs)
    real_loops = set()
    for comp_name in computations_dict:
        real_loops.update(computations_dict[comp_name]['iterators'])
        
    #building loop tensor
    loops_representation_list = []
    loops_indices_dict = dict()
    loop_index=0
    for loop_name in program_json['iterators']:
        if not (loop_name in real_loops): # this removes the iterators that are only used for decraling inputs
            continue
        loop_representation=[]
        loop_dict = program_json['iterators'][loop_name]
        # upper and lower bound
        loop_representation.append(loop_dict['upper_bound'])
        loop_representation.append(loop_dict['lower_bound'])
        if loop_schedules_dict[loop_name]['unfused']:
            loop_representation.append(1) #unfused True
        else:
            loop_representation.append(0) #unfused False
        if loop_schedules_dict[loop_name]['interchanged']:
            loop_representation.append(1) #interchanged True
        else:
            loop_representation.append(0) #interchanged False     
        if loop_schedules_dict[loop_name]['skewed']:
            loop_representation.append(1) #skewed True
            loop_representation.append(loop_schedules_dict[loop_name]['skew_factor']) #skew factor
            loop_representation.append(loop_schedules_dict[loop_name]['skew_extent']) #skew extent
        else:
            loop_representation.append(0) # skewed false
            loop_representation.append(0) # factor
            loop_representation.append(0) # extent
        if loop_schedules_dict[loop_name]['parallelized']:
            loop_representation.append(1) #parallelized True
        else:
            loop_representation.append(0) # parallelized false
        if loop_schedules_dict[loop_name]['tiled']:
            loop_representation.append(1) #tiled True
            loop_representation.append(loop_schedules_dict[loop_name]['tile_factor']) #tile factor
        else:
            loop_representation.append(0) #tiled False
            loop_representation.append(0) #tile factor 0
        # TODO: check if unroll representation should be moved to comp vector instead of loop vector
        if loop_schedules_dict[loop_name]['unrolled']:
            loop_representation.append(1) #unrolled True
            loop_representation.append(loop_schedules_dict[loop_name]['unroll_factor']) #unroll factor
        else:
            loop_representation.append(0) #unrolled False
            loop_representation.append(0) #unroll factor 0
        # adding log(x+1) of the loop representation
        loop_log_rep = list(np.log1p(loop_representation))
        loop_representation.extend(loop_log_rep)
        loops_representation_list.append(loop_representation)    
        loops_indices_dict[loop_name]=loop_index
        loop_index+=1
            
     
    def update_tree_atributes(node):     
        node['loop_index'] = torch.tensor(loops_indices_dict[node['loop_name'][:3]]).to(train_device)
        if node['computations_list']!=[]:
            node['computations_indices'] = torch.tensor([indices_dict[comp_name] for comp_name in node['computations_list']]).to(train_device)
            node['has_comps'] = True
        else:
            node['has_comps'] = False
        for child_node in node['child_list']:
            update_tree_atributes(child_node)
        return node
    
    tree_annotation = copy.deepcopy(schedule_json['tree_structure']) #to avoid altering the original tree from the json
    prog_tree = update_tree_atributes(tree_annotation) 
    
    loops_tensor = torch.unsqueeze(torch.FloatTensor(loops_representation_list),0)#.to(device)
    computations_tensor = torch.unsqueeze(torch.FloatTensor(program_representation),0)#.to(device)     

    return prog_tree, computations_tensor, loops_tensor


#################################################

# def get_tree_footprint(tree):
#     footprint='<BL'+str(int(tree['loop_index']))
#     if tree['has_comps']:
#         footprint+='['
#         for idx in tree['computations_indices']:
#             footprint+='CI'+str(int(idx))
#         footprint+=']'
#     for child in tree['child_list']:
#         footprint+= get_tree_footprint(child)
#     footprint+='EL'+str(int(tree['loop_index']))+'>'
#     return footprint
    
class Dataset_LI():
    def __init__(self, dataset_MC_filename, dataset_SC_filename, max_batch_size, filter_func=None, filter_func_MC=None,filter_func_SC=None, transform_func=None):
        super().__init__()
        
        self.X = []
        self.Y = []
        self.batched_program_names = []
        self.batched_schedule_names = []
        self.batched_exec_time = []
        self.nb_nan=0
        self.nb_long_access=0
        self.batches_dict=dict()
            
        
        if(dataset_MC_filename!=None):
        
            #loading multi computation programs

            self.dataset_MC_name=dataset_MC_filename

            if dataset_MC_filename.endswith('json'):
                with open(dataset_MC_filename, 'r') as f:
                    dataset_MC_str = f.read()
                self.programs_dict_MC = json.loads(dataset_MC_str)
            elif dataset_MC_filename.endswith('pkl'):
                with open(dataset_MC_filename, 'rb') as f:
                    self.programs_dict_MC = pickle.load(f)

            if (filter_func_MC==None):
                filter_func_MC = lambda x : True
            if (transform_func==None):
                transform_func = lambda x : x


            for function_name in tqdm(self.programs_dict_MC):
#                 print("MC : ", function_name)
                if (np.min(self.programs_dict_MC[function_name]['schedules_list'][0]['execution_times'])<0): #if less than x ms
                    continue

                program_json = self.programs_dict_MC[function_name]['program_annotation']
                program_exec_time = self.programs_dict_MC[function_name]['initial_execution_time']
                loops = shared_loop_nest(program_json)

                schedules_dict = {}   #####
                explored_schedules = {}

                for schedule_index in range(len(self.programs_dict_MC[function_name]['schedules_list'])):
                    sched_str = self.programs_dict_MC[function_name]['schedules_list'][schedule_index]["sched_str"]
                    schedule_json = self.programs_dict_MC[function_name]['schedules_list'][schedule_index]

                    if (not filter_func_MC(sched_str)) or (not legal_LI(sched_str, schedule_index, loops)):
                        continue

                    sched_exec_time = np.min(schedule_json['execution_times'])
                    self.programs_dict_MC[function_name]['schedules_list'][schedule_index]['speedup'] = max(program_exec_time / sched_exec_time,0.01) #speedup clipping
                    if ((np.isnan(self.programs_dict_MC[function_name]['schedules_list'][schedule_index]['speedup']))
                         or(self.programs_dict_MC[function_name]['schedules_list'][schedule_index]['speedup']==0)): #Nan value means the schedule didn't run, zero values means exec time<1 micro-second, skip them
                        self.nb_nan+=1
                        continue


                    #look for the parent schedule
                    scheduleP_str = re.sub("I\(\{[C0-9,]+\},L[0-9]+,L[0-9]+\)", '', sched_str)
#                     print(scheduleP_str, sched_str)
                    if scheduleP_str in explored_schedules.keys():  #parent schedule exist, check if this is better
                        schedulePID = explored_schedules[scheduleP_str]
            
                        #for k best
                        ind_LI = index_interchage(sched_str, len(program_json['iterators']))
                        if sched_exec_time < schedules_dict[schedulePID]['all'][ind_LI]:
                            schedules_dict[schedulePID]['all'][ind_LI] = sched_exec_time
                    
                        if sched_exec_time >= schedules_dict[schedulePID]['best']: # not the best
                            continue
                        else :
                            schedules_dict[schedulePID]['best'] = sched_exec_time
                            schedules_dict[schedulePID]['sched'] = schedule_index
#                             sched_dict = {'best':sched_exec_time, 'sched':schedule_index}
#                             schedules_dict[schedulePID] = sched_dict
                        

                    else: # parent schedule is new 
                        scheduleP, schedulePID = get_sched_by_string(scheduleP_str, self.programs_dict_MC[function_name]['schedules_list'])
#                         print(scheduleP, schedulePID )
                        explored_schedules[scheduleP_str]=schedulePID  # discoverd a new schedule

                        schedP_exec_time = np.min(scheduleP['execution_times'])

                        if sched_exec_time >= schedP_exec_time: # parent is best
                            schedP_dict = {'best':schedP_exec_time, 'sched':schedulePID}
                            schedules_dict[schedulePID] = schedP_dict
                        else : # schedule is best
                            sched_dict = {'best':sched_exec_time, 'sched':schedule_index}
                            schedules_dict[schedulePID] = sched_dict
                        
                        #for k_best
                        output = np.zeros(106,dtype=np.float64) + np.inf # because, a priori, all of them are very bad and does not execute well
                        output[0] = schedP_exec_time
                        ind_LI = index_interchage(sched_str, len(program_json['iterators']))
                        output[ind_LI] = sched_exec_time
                        schedules_dict[schedulePID]['all'] = output


                #for each fused program or the original pgme
                for elem in schedules_dict.items():
                    sched_json = self.programs_dict_MC[function_name]['schedules_list'][elem[0]]
                    try:
                        tree, comps_tensor, loops_tensor = get_representation_LI(program_json, sched_json) ######
                    except LargeAccessMatices:
                        self.nb_long_access +=1
                        continue   

                    # for each datapoint append its best LI

                    tree_footprint=get_tree_footprint(tree) 
                    self.batches_dict[tree_footprint] = self.batches_dict.get(tree_footprint,{'tree':tree,'comps_tensor_list':[],'loops_tensor_list':[],'program_names_list':[],'sched_names_list':[],'speedups_list':[],'exec_time_list':[], 'output':[]}) 
                    self.batches_dict[tree_footprint]['comps_tensor_list'].append(comps_tensor)
                    self.batches_dict[tree_footprint]['loops_tensor_list'].append(loops_tensor)
                    self.batches_dict[tree_footprint]['sched_names_list'].append(elem[0])
                    self.batches_dict[tree_footprint]['program_names_list'].append(function_name)

                    speedup = max(program_exec_time / elem[1]['best'] , 0.01) #speedup clipping
                    self.batches_dict[tree_footprint]['speedups_list'].append(speedup)  
                    self.batches_dict[tree_footprint]['exec_time_list'].append(elem[1]['best'])

                    best_schedule_str = self.programs_dict_MC[function_name]['schedules_list'][elem[1]['sched']]['sched_str'] #does it contains only LI ?
                    
                    #Y
                    output= elem[1]['all']
                    output = output[0] / output 
        #             print("-----------------")
        #             print(output)
                    order = np.flip(np.argsort(output, -1)) #highest to smallest
        #             print(order)
                    y = np.zeros(106 * k,dtype=np.float64)
                    for i in range(k):
                        if output[order[i]] == 0:
                            y[106*i] = 1 # No LI
                            continue
                        y[106*i+order[i]] = 1 #define the output, using the best order we have in order % speedups. Because,it returns the indexes of the best options, and thus, the ones that should be put in 1.

                    self.batches_dict[tree_footprint]['output'].append(torch.tensor(y)) 

     
        #loading single computation programs
        if(dataset_SC_filename!=None):
            self.dataset_SC_name=dataset_SC_filename

            if dataset_SC_filename.endswith('json'):
                with open(dataset_SC_filename, 'r') as f:
                    dataset_SC_str = f.read()
                self.programs_dict_SC = json.loads(dataset_SC_str)
            elif dataset_SC_filename.endswith('pkl'):
                with open(dataset_SC_filename, 'rb') as f:
                    self.programs_dict_SC = pickle.load(f)

            if (filter_func_SC==None):
                filter_func_SC = lambda x : True
            if (transform_func==None):
                transform_func = lambda x : x


            for function_name in tqdm(self.programs_dict_SC):
                if (np.min(self.programs_dict_SC[function_name]['schedules_list'][0]['execution_times'])<0): #if less than x ms
                    continue
                program_json = self.programs_dict_SC[function_name]['program_annotation']
                program_exec_time = self.programs_dict_SC[function_name]['initial_execution_time']
                Parent_sched_json = self.programs_dict_SC[function_name]['schedules_list'][0]   # get the parent schedule


                try:
                    tree, comps_tensor, loops_tensor = get_representation_LI(program_json, Parent_sched_json) ######
                except LargeAccessMatices:
                    self.nb_long_access +=1
                    continue        

                #######
                min_sch_time = np.inf   
                output = np.zeros(106,dtype=np.float64) + np.inf # because, a priori, all of them are very bad and does not execute well
                
                for schedule_index in range(len(self.programs_dict_SC[function_name]['schedules_list'])):
                    sched_str = sched_json_to_sched_str( self.programs_dict_SC[function_name]['schedules_list'][schedule_index] )
                    if (not filter_func_SC(sched_str)):
                        continue  

                    sched_exec_time = np.min(self.programs_dict_SC[function_name]['schedules_list'][schedule_index]['execution_times'])
                    self.programs_dict_SC[function_name]['schedules_list'][schedule_index]['speedup'] = max(program_exec_time / sched_exec_time,0.01) #speedup clipping
                    if ((np.isnan(self.programs_dict_SC[function_name]['schedules_list'][schedule_index]['speedup']))
                         or(self.programs_dict_SC[function_name]['schedules_list'][schedule_index]['speedup']==0)): #Nan value means the schedule didn't run, zero values means exec time<1 micro-second, skip them
                        self.nb_nan+=1
                        continue

                    #if another schedule used the same interchange before
                    if output[index_interchage(sched_str,15)] > sched_exec_time:
                        output[index_interchage(sched_str,15)] = sched_exec_time #position all the execution times
                
                    if sched_exec_time >= min_sch_time:
                            continue   # we only keep the best loop interchange for a given program
                    min_sch_time = sched_exec_time
                    #best_schedule_str = self.programs_dict_SC[function_name]['schedules_list'][schedule_index]['sched_str']
                    best_schedule_str = sched_json_to_sched_str( self.programs_dict_SC[function_name]['schedules_list'][schedule_index] )
               ########
                
            
                # for each function append its best LI

                tree_footprint=get_tree_footprint(tree) 
                self.batches_dict[tree_footprint] = self.batches_dict.get(tree_footprint,{'tree':tree,'comps_tensor_list':[],'loops_tensor_list':[],'program_names_list':[],'sched_names_list':[],'speedups_list':[],'exec_time_list':[], 'output':[]}) 
                self.batches_dict[tree_footprint]['comps_tensor_list'].append(comps_tensor)
                self.batches_dict[tree_footprint]['loops_tensor_list'].append(loops_tensor)
                #self.batches_dict[tree_footprint]['sched_names_list'].append(schedule_name)  ommit it bcs the schedule is always the 0th schedule ==> not significant
                self.batches_dict[tree_footprint]['program_names_list'].append(function_name)
                self.batches_dict[tree_footprint]['speedups_list'].append(self.programs_dict_SC[function_name]['schedules_list'][0]['speedup'])  ######
                self.batches_dict[tree_footprint]['exec_time_list'].append(program_exec_time)   ######
                
                #Y
                output = output[0] / output 
                order = np.flip(np.argsort(output, -1)) #highest to smallest
                y = np.zeros(106 * k,dtype=np.float64)
                for i in range(k):
                    if output[order[i]] == 0:
                        y[106*i] = 1 # No LI
                        continue
                    y[106*i+order[i]] = 1 #define the output, using the best order we have in order % speedups. Because,it returns the indexes of the best options, and thus, the ones that should be put in 1.

                self.batches_dict[tree_footprint]['output'].append(torch.tensor(y))

        
 
        storing_device = store_device
        for tree_footprint in self.batches_dict:
            for chunk in range(0,len(self.batches_dict[tree_footprint]['program_names_list']),max_batch_size):  #####
                if storing_device.type=='cuda': # Check GPU memory in order to avoid Out of memory error
                    if ((torch.cuda.memory_allocated(storing_device.index)/torch.cuda.get_device_properties(storing_device.index).total_memory)>0.80):
                        print('GPU memory on '+str(storing_device)+' nearly full, switching to CPU memory')
                        storing_device = torch.device('cpu')
                #self.batched_schedule_names.append(self.batches_dict[tree_footprint]['sched_names_list'][chunk:chunk+max_batch_size])   #####
                self.batched_program_names.append(self.batches_dict[tree_footprint]['program_names_list'][chunk:chunk+max_batch_size])
                self.batched_exec_time.append(self.batches_dict[tree_footprint]['exec_time_list'][chunk:chunk+max_batch_size])
                self.X.append( ( self.batches_dict[tree_footprint]['tree'],
                               torch.cat(self.batches_dict[tree_footprint]['comps_tensor_list'][chunk:chunk+max_batch_size], 0).to(storing_device),
                               torch.cat(self.batches_dict[tree_footprint]['loops_tensor_list'][chunk:chunk+max_batch_size], 0).to(storing_device) ) )
                
                temp_tens = torch.cat(self.batches_dict[tree_footprint]['output'][chunk:chunk+max_batch_size],0)
                dim_temp_tens = int(temp_tens.shape[0] / (106 * 5))
#                 print(temp_tens.shape)
                self.Y.append(torch.reshape(temp_tens, (dim_temp_tens,106 *5)).to(storing_device)) ###### was (dim_temp_tens,1,106))
                
                if len(self.X) != len(self.Y):
                    print(len(self.X[-1]), len(self.Y[-1]))   # to look here 
                    print(type(self.X),len(self.X), type(self.Y),len(self.Y),type(self.X[0][1]),len(self.X[0][1]))
                    print("stop")
                                                
        print(f'Number of batches {len(self.Y)}')
        if self.nb_long_access>0:
            print('Number of batches dropped due to too much memory accesses:' +str(self.nb_long_access))
            
            
    def __getitem__(self, index):
        if isinstance(index, slice):
            start, stop, step = index.indices(len(self))
            return [self[i] for i in range(start, stop, step)]
        elif isinstance(index, int):
            return self.X[index], self.Y[index] 

    def __len__(self):
        return len(self.Y)

                    
def load_merge_data(train_val_MC,train_val_SC,split_ratio=None, max_batch_size=2048, filter_func_MC=None, filter_func_SC=None):
    full_dataset = Dataset_LI(train_val_MC, train_val_SC, max_batch_size,filter_func_MC=filter_func_MC, filter_func_SC=filter_func_SC)
    if split_ratio == None:
        split_ratio=0.2
    if split_ratio > 1 : # not a ratio a number of batches
        validation_size = split_ratio
    else:
        validation_size = int(split_ratio * len(full_dataset))
    indices = list(range(len(full_dataset)))
    random.Random(42).shuffle(indices)
    val_batches_indices, train_batches_indices = indices[:validation_size],\
                                               indices[validation_size:]
    val_batches_list=[]
    train_batches_list=[]
    for i in val_batches_indices:
        val_batches_list.append(full_dataset[i])
    for i in train_batches_indices:
        train_batches_list.append(full_dataset[i])
    print("Data loaded")
    print("Sizes: "+str((len(val_batches_list),len(train_batches_list)))+" batches")
    return full_dataset, val_batches_list, val_batches_indices, train_batches_list, train_batches_indices

class Model_Recursive_LSTM_LI(nn.Module):
    def __init__(self, input_size, comp_embed_layer_sizes=[600, 350, 200, 180], drops=[0.225, 0.225, 0.225, 0.225], output_size=1):
        super().__init__()
        embedding_size = comp_embed_layer_sizes[-1]
        regression_layer_sizes = [embedding_size] + comp_embed_layer_sizes[-2:]
        concat_layer_sizes = [embedding_size*2+24] + comp_embed_layer_sizes[-2:]
        comp_embed_layer_sizes = [input_size] + comp_embed_layer_sizes
        self.comp_embedding_layers = nn.ModuleList()
        self.comp_embedding_dropouts= nn.ModuleList()
        self.regression_layers = nn.ModuleList()
        self.regression_dropouts= nn.ModuleList()
        self.concat_layers = nn.ModuleList()
        self.concat_dropouts= nn.ModuleList()
        for i in range(len(comp_embed_layer_sizes)-1):
            self.comp_embedding_layers.append(nn.Linear(comp_embed_layer_sizes[i], comp_embed_layer_sizes[i+1], bias=True))
            nn.init.xavier_uniform_(self.comp_embedding_layers[i].weight)
            self.comp_embedding_dropouts.append(nn.Dropout(drops[i]))
        for i in range(len(regression_layer_sizes)-1):
            self.regression_layers.append(nn.Linear(regression_layer_sizes[i], regression_layer_sizes[i+1], bias=True))
            nn.init.xavier_uniform_(self.regression_layers[i].weight)
            self.regression_dropouts.append(nn.Dropout(drops[i]))
        for i in range(len(concat_layer_sizes)-1):
            self.concat_layers.append(nn.Linear(concat_layer_sizes[i], concat_layer_sizes[i+1], bias=True))
#             nn.init.xavier_uniform_(self.concat_layers[i].weight)
            nn.init.zeros_(self.concat_layers[i].weight)
            self.concat_dropouts.append(nn.Dropout(drops[i]))
        self.predict = nn.Linear(regression_layer_sizes[-1], output_size, bias=True)
        nn.init.xavier_uniform_(self.predict.weight)
        self.ELU=nn.ELU()
        self.no_comps_tensor = nn.Parameter(nn.init.xavier_uniform_(torch.zeros(1, embedding_size)))
#         self.ELU = nn.Tanh()
        self.no_nodes_tensor = nn.Parameter(nn.init.xavier_uniform_(torch.zeros(1, embedding_size)))
        self.comps_lstm = nn.LSTM(comp_embed_layer_sizes[-1], embedding_size, batch_first=True)
        self.nodes_lstm = nn.LSTM(comp_embed_layer_sizes[-1], embedding_size, batch_first=True)
        
    def get_hidden_state(self, node, comps_embeddings, loops_tensor):
        nodes_list = []
        for n in node['child_list']:
            nodes_list.append(self.get_hidden_state(n, comps_embeddings,loops_tensor))
        if (nodes_list != []):
            nodes_tensor = torch.cat(nodes_list, 1) 
            lstm_out, (nodes_h_n, nodes_c_n) = self.nodes_lstm(nodes_tensor)
            nodes_h_n = nodes_h_n.permute(1,0,2)
        else:       
            nodes_h_n = torch.unsqueeze(self.no_nodes_tensor, 0).expand(comps_embeddings.shape[0], -1, -1)
        if (node['has_comps']):
            selected_comps_tensor = torch.index_select(comps_embeddings, 1, node['computations_indices'])
            lstm_out, (comps_h_n, comps_c_n) = self.comps_lstm(selected_comps_tensor) 
            comps_h_n = comps_h_n.permute(1,0,2)
        else:
            comps_h_n = torch.unsqueeze(self.no_comps_tensor, 0).expand(comps_embeddings.shape[0], -1, -1)
        selected_loop_tensor = torch.index_select(loops_tensor,1,node['loop_index'])
        x = torch.cat((nodes_h_n, comps_h_n, selected_loop_tensor),2)
        for i in range(len(self.concat_layers)):
            x = self.concat_layers[i](x)
            x = self.concat_dropouts[i](self.ELU(x))
        return x  

    def forward(self, tree_tensors):
        tree, comps_tensor, loops_tensor = tree_tensors
        #computation embbedding layer
        x = comps_tensor
        for i in range(len(self.comp_embedding_layers)):
            x = self.comp_embedding_layers[i](x)
            x = self.comp_embedding_dropouts[i](self.ELU(x))  
        comps_embeddings = x
        #recursive loop embbeding layer
        prog_embedding = self.get_hidden_state(tree, comps_embeddings, loops_tensor)
        #regression layer
        x = prog_embedding
        for i in range(len(self.regression_layers)):
            x = self.regression_layers[i](x)
            x = self.regression_dropouts[i](self.ELU(x))
        out = self.predict(x)
#         print(out[:,0,:].shape)
        return out[:,0,:]  #(out[:,0])#self.ELU(out[:,0,0])       nb_elem/batch  nb_comp 15
    
    
def get_results_df_LI(dataset, batches_list, indices, model, log=False):   
    df = pd.DataFrame()
    model.eval()
    torch.set_grad_enabled(False)
    all_outputs=[]
    all_labels=[]
    prog_names=[]
    #sched_names=[]
    exec_times=[]
    soft = nn.Softmax(-1)

    for k, (inputs, labels) in tqdm(list(enumerate(batches_list))):
        original_device = labels.device
        inputs=(inputs[0], inputs[1].to(train_device), inputs[2].to(train_device))
        labels=labels.to(train_device)
        outputs = model(inputs)
        assert outputs.shape == labels.shape
        
        
        outputs_soft = soft(outputs[...,0:106]) # special case
        for i in range(1,5):
            outputs_soft = torch.cat((outputs_soft,soft(outputs[...,106*i:106*(i+1)])),-1)
        assert outputs_soft.shape == labels.shape
        #verified: outputs_softmax has the same shape
        all_outputs.append(outputs_soft) ##fixed
        all_labels.append(labels)

        for j, prog_name in enumerate(dataset.batched_program_names[indices[k]]):   #####
            #sched_names.append(sched_name)
            prog_names.append(dataset.batched_program_names[indices[k]][j])
            exec_times.append(dataset.batched_exec_time[indices[k]][j])
        inputs=(inputs[0], inputs[1].to(original_device), inputs[2].to(original_device))
        labels=labels.to(original_device)
        
    preds = torch.cat(all_outputs).cpu().detach().numpy() #because it is modified above to be a tensor
    targets = torch.cat(all_labels).cpu().detach().numpy()
    
                                  
    assert preds.shape == targets.shape 
    df['name'] = prog_names
    df['exec_time'] = exec_times
    df['prediction'] = list(format_output(preds)[:,:]) #MISTAKES 
    df['target'] = list(targets[:,:])
    return df


In [8]:
#####################################
# functions for LI 
def pos(a,depth):  # calculate the position of the interchange in the output vector
    if(a==0):
        return 0
    elif (a==1):
        return depth-1
    else:
        return pos(a-1,depth)+depth-a

def encode_interchage(str_sched,program_depth): #encode the output vector to get '1' in the right LI, '0' elsewhere
    output = np.zeros(106,dtype=int) # 106 = 2 C 15 + 1
    str_interchange = re.findall('I\(.*\)', str_sched)  # get the LI string
    if str_interchange==[]:
        output[0] = 1
    else:
        m = re.findall(r'\d+', str_interchange[0])  # get the number of the loops
        a = min(int(m[-2]),int(m[-1]))
        b = max(int(m[-2]),int(m[-1]))
        i2 = b-a
        i1 = pos(a,15)
        output[i1+i2]=1      
    return output

def index_interchage(str_sched,program_depth): #encode the output vector to get '1' in the right LI, '0' elsewhere
    output = np.zeros(106,dtype=int) # 106 = 2 C 15 + 1
    str_interchange = re.findall('I\(.*\)', str_sched)  # get the LI string
    if str_interchange==[]:
        return(0)
    else:
        m = re.findall(r'\d+', str_interchange[0])  # get the number of the loops
        a = min(int(m[-2]),int(m[-1]))
        b = max(int(m[-2]),int(m[-1]))
        i2 = b-a
        i1 = pos(a,15)
        return(i1+i2)      

    
    
def get_sched_by_string(sched_str,schedules):
    for schedID in range(len(schedules)):
        sched = schedules[schedID]
        schedExp_str = sched['sched_str']
        if schedExp_str == sched_str: # found the right schedule
            return sched, schedID
        
def get_results_times_function(vector,schedules):
    
    sched_str = "I(L"
    np_vect= np.array(vector)
    indices = np.where(np_vect==1)[0]
    if len(indices) != 0: # check if a LI is applied
        sched_str = sched_str + str(indices[0]) + ",L" + str(indices[1]) + ")"
    else :
        sched_str = ""
        
    sched = get_sched_by_string(sched_str,schedules)
    if sched != None :
        sched_exec_time = np.min(sched['execution_times']) 
        return sched_exec_time
    else :
        return -1
    

def format_output(arr):  ##
#     indices = np.argsort(arr)[:,-1:] # shape of (#elems, 1)
    pred = np.zeros(arr.shape)
    for b in range(k):
        indices = np.argmax(arr[...,106*b:106*(b+1)], -1)
        for i in range(indices.shape[0]):
            pred[i,106*b + indices[i]] = 1
    return pred

#####################################
# functions for SC 

def speedup_clip(speedup):
    if speedup<0.01:
        speedup = 0.01
    return speedup

def sched_json_to_sched_str(sched_json): # Works only for 1 comp programs
    orig_loop_nest = []
    orig_loop_nest.append(sched_json['tree_structure']['loop_name'])
    child_list = sched_json['tree_structure']['child_list']
    while len(child_list)>0:
        child_loop = child_list[0]
        orig_loop_nest.append(child_loop['loop_name'])
        child_list = child_loop['child_list']
        
    comp_name = [n for n in sched_json.keys() if not n in ['unfuse_iterators','tree_structure','execution_times']][0]
    schedule = sched_json[comp_name]
    transf_loop_nest = orig_loop_nest
    sched_str = ''
    
    if schedule['interchange_dims']:
        first_dim_index = transf_loop_nest.index(schedule['interchange_dims'][0])
        second_dim_index = transf_loop_nest.index(schedule['interchange_dims'][1])
        sched_str+='I(L'+str(first_dim_index)+',L'+str(second_dim_index)+')'
        transf_loop_nest[first_dim_index], transf_loop_nest[second_dim_index] = transf_loop_nest[second_dim_index], transf_loop_nest[first_dim_index]
    if schedule['skewing']:
        first_dim_index = transf_loop_nest.index(schedule['skewing']['skewed_dims'][0])
        second_dim_index = transf_loop_nest.index(schedule['skewing']['skewed_dims'][1])
        first_factor = schedule['skewing']['skewing_factors'][0]
        second_factor = schedule['skewing']['skewing_factors'][1]
        sched_str+='S(L'+str(first_dim_index)+',L'+str(second_dim_index)+','+str(first_factor)+','+str(second_factor)+')'
    if schedule['parallelized_dim']:
        dim_index = transf_loop_nest.index(schedule['parallelized_dim'])
        sched_str+='P(L'+str(dim_index)+')'
    if schedule['tiling']:
        if schedule['tiling']['tiling_depth']==2:
            first_dim = schedule['tiling']['tiling_dims'][0]
            second_dim = schedule['tiling']['tiling_dims'][1]
            first_dim_index = transf_loop_nest.index(first_dim)
            second_dim_index = transf_loop_nest.index(second_dim)
            first_factor = schedule['tiling']['tiling_factors'][0]
            second_factor = schedule['tiling']['tiling_factors'][1]
            sched_str+='T2(L'+str(first_dim_index)+',L'+str(second_dim_index)+','+str(first_factor)+','+str(second_factor)+')'
            i = transf_loop_nest.index(first_dim)
            transf_loop_nest[i:i+1]=first_dim+'_outer', second_dim+'_outer'
            i = transf_loop_nest.index(second_dim)
            transf_loop_nest[i:i+1]=first_dim+'_inner', second_dim+'_inner'
        else: #tiling depth == 3
            first_dim = schedule['tiling']['tiling_dims'][0]
            second_dim = schedule['tiling']['tiling_dims'][1]
            third_dim = schedule['tiling']['tiling_dims'][2]
            first_dim_index = transf_loop_nest.index(first_dim)
            second_dim_index = transf_loop_nest.index(second_dim)
            third_dim_index = transf_loop_nest.index(third_dim)
            first_factor = schedule['tiling']['tiling_factors'][0]
            second_factor = schedule['tiling']['tiling_factors'][1]
            third_factor = schedule['tiling']['tiling_factors'][2]
            sched_str+='T3(L'+str(first_dim_index)+',L'+str(second_dim_index)+',L'+str(third_dim_index)+','+str(first_factor)+','+str(second_factor)+','+str(third_factor)+')'
            i = transf_loop_nest.index(first_dim)
            transf_loop_nest[i:i+1]=first_dim+'_outer', second_dim+'_outer', third_dim+'_outer'
            i = transf_loop_nest.index(second_dim)
            transf_loop_nest[i:i+1]=first_dim+'_inner', second_dim+'_inner', third_dim+'_inner'
            transf_loop_nest.remove(third_dim)
    if schedule['unrolling_factor']:
        dim_index = len(transf_loop_nest)-1
        dim_name =transf_loop_nest[-1]
        sched_str+='U(L'+str(dim_index)+','+schedule['unrolling_factor']+')'
        transf_loop_nest[dim_index:dim_index+1] = dim_name+'_Uouter', dim_name+'_Uinner'
    
    return sched_str

#####################################
# functions for MC

def shared_loop_nest(program_json):
    stop = False
    loop_nest = []
    iterators = program_json['iterators']
    j = 0
    for i in list(iterators.keys()) : 
        if not stop : 
            iterator = iterators[i]
            if iterator['child_iterators'] != [] :  #in the middle of the tree
                if iterator['computations_list'] != [] :  #has some computations
                    stop = True
                else:
                    if len( iterator['child_iterators'] ) >= 2 :
                        stop = True
            if stop == False :
                loop_nest.append( "i"+str(j) )
        else : 
            return loop_nest
        j = j + 1
    return loop_nest

def legal_LI(schedule_str, sched_ind, loops):
    
    if(schedule_str==""):
        return True
    else :
        regex1 = "I\(\{[C0-9,]+\},L[0-9]+,L[0-9]+\)$" 
        LI = re.findall(regex1, schedule_str)  # get the LI string
        
        m = re.findall(r'\d+', LI[0])  # get the loops
        L1 = 'i' + m[-2]
        L2 = 'i' + m[-1]
        if (L1 in loops) and (L2 in loops):
            return True
        else:
            return False
        

In [81]:
#restore the string of the LI from the index of the LI model output vector
def get_LI_ordered(output, k = 5):
    LIs = []
    taken = []
    for i in range(k):
        ind = np.argmax(output[...,106*i:106*(i+1)]) # return the indice of the chosen LI
        if ind in taken:
            continue
        taken.append(ind)
        if ind == 0:
            LIs.append("")
        else:
            for i in range(15):
                if (ind < (15 - i)):
                    str2 = "I(L" + str(i) + ",L" + str(ind+i)+ ")"
                    str2.strip()
                    if str2 not in LIs:
                        LIs.append(str2)
                    break
                else:
                    ind= ind - 15 + i + 1
    return LIs